In [1]:
import pandas as pd
import xlsxwriter
import datetime
import numpy as np
import calendar
import xlrd

In [26]:
def setup_data_for_cr_calculations(data):
    group_number_of_complaints = data.loc[data['Measure Names'] == '# of Complaints'] \
        .groupby(
        ['SC Business Segment', 'SC Modality', 'SC Product Segment', 'SC Product', 'By Time Label', 'Year', 'Month',
         'Measure Names'])['Measure Values'].sum()
    group_ib = data.loc[data['Measure Names'] == 'Average IB Over Time'] \
        .groupby(['SC Business Segment', 'SC Modality', 'SC Product Segment','SC Product', 'By Time Label', 'Year', 'Month',
                  'Measure Names'])['Measure Values'].sum()

    final_data = pd.DataFrame(group_number_of_complaints).reset_index()
    final_data = final_data.append(pd.DataFrame(group_ib).reset_index())
    return final_data

def setup_data_for_cr_calculations_pqm(data):
    data['Measure Names'] = '# of Complaints'
    group_number_of_complaints = data.loc[data['Measure Names'] == '# of Complaints'] \
        .groupby(
        ['SC Business Segment', 'SC Modality', 'SC Product Segment', 'SC Product', 'By Time Label', 'Year', 'Month',
         'Measure Names'])['Measure Values'].sum()
        

    final_data = pd.DataFrame(group_number_of_complaints).reset_index()
    #final_data = final_data.append(pd.DataFrame(group_ib).reset_index())
    return final_data
    


def setup_scorecard_structure(data):
    #print(data.columns.values)
    current_year = datetime.datetime.now().year
    columns = []
    columns.append('Product Quality Scorecard')
    columns.append('Business')
    columns.append('Modality')
    columns.append('Product Segment')
    start_year = 2016
    while start_year < current_year + 1:
        if start_year != current_year:
            columns.append(str(start_year) + ' Total')
        if start_year == current_year or start_year == current_year - 1:
            columns.append(str(start_year) + ' YTD')
        start_year = start_year + 1
    score_card = pd.DataFrame(columns=columns)
    index = 0
    score_card.loc[index, 'Product Quality Scorecard'] = 'GEHC'
    data['SC Product'].fillna("", inplace=True)
    for business in data['SC Business Segment'].sort_values(ascending=False).unique():
        index = index + 1
        score_card.loc[index, 'Product Quality Scorecard'] = business
        for modality in np.unique(data.loc[data['SC Business Segment'] == business]['SC Modality'].values):
            index = index + 1
            score_card.loc[index, 'Product Quality Scorecard'] = modality
            score_card.loc[index, 'Business'] = business
            for product_seg in np.unique(data.loc[data['SC Modality'] == modality]['SC Product Segment'].values):
                index = index + 1
                score_card.loc[index, 'Product Quality Scorecard'] = product_seg
                score_card.loc[index, 'Modality'] = modality
                score_card.loc[index, 'Business'] = business
                for product in np.unique(
                        data.loc[(data['SC Modality'] == str(modality)) & (
                                    data['SC Product Segment'] == str(product_seg))]['SC Product'].values):
                    index = index + 1
                    score_card.loc[index, 'Product Quality Scorecard'] = product
                    score_card.loc[index, 'Product Segment'] = product_seg
                    score_card.loc[index, 'Modality'] = modality
                    score_card.loc[index, 'Business'] = business

        score_card['Business'].fillna('All', inplace=True)
        score_card['Modality'].fillna('All', inplace=True)
        score_card['Product Segment'].fillna('All', inplace=True)

    return score_card

def setup_sii_sc_hierarchy(data, sii_hierarchy, modality_lookup, metric):
    modality_metric_column = ''
    sub_modality_metric_column = ''
    config_modality_column = ''
    a_level_modality_column = ''
    if metric == 'OTD':
        modality_metric_column = 'SII Modality + DV Modality'
        config_modality_column = 'DV Modality'
        sub_modality_metric_column = 'SIISUBMODALITY'
        a_level_modality_column = 'a_level_modality_code'
        abbr_to_num = {name: num for num, name in enumerate(calendar.month_name) if num}
        month_names = data['Month'].unique()
        for month in month_names:
            data.loc[data['Month'] == month, 'Month'] = abbr_to_num[month]
    elif metric == 'OTI':
        modality_metric_column = 'SII Modality or OTI Modality'
        sub_modality_metric_column = 'SUB_MODLTY_CD'
        a_level_modality_column = 'a_level_modality_code'
        config_modality_column = 'Modality (SFDC)'
        time_labels = data['By Time Label'].unique()
        for time_label in time_labels:
            data.loc[data['By Time Label'] == time_label, "Year"] = time_label.split('M')[0]
            data.loc[data['By Time Label'] == time_label, "Month"] = time_label.split('M')[1]
        data['Year'] = data['Year'].astype(int)
        data['Month'] = data['Month'].astype(int)
    elif metric == 'CSO':
        modality_metric_column = 'SII Modality or CSO Modality'
        a_level_modality_column = 'a_level_modality'
        sub_modality_metric_column = 'SUBMODALITY'
        config_modality_column = 'Modality Group (SFDC)'
        #print(data.columns)
        time_labels = data['Snapshot FW'].unique()
        for time_label in time_labels:
            data.loc[data['Snapshot FW'] == time_label, "Year"] = 2000 + int(time_label.split('-')[0])
            data.loc[data['Snapshot FW'] == time_label, 'Week'] = int(time_label.split('-')[1])
        data['Year'] = data['Year'].astype(int)
        data['Week'] = data['Week'].astype(int)
    elif metric == 'ELF':
        modality_metric_column = 'SII Modality '
        a_level_modality_column = 'A Level Modality Code'
        sub_modality_metric_column = 'Sub-Modality'
        config_modality_column = 'Adjusted Modality Group'
        time_labels = data['Time Label'].unique()
        for time_label in time_labels:
            data.loc[data[
                         'Time Label'] == time_label,
                     "Year of service_close_date"] = \
                time_label.split('M')[0]
            data.loc[data['Time Label'] == time_label, 'Month Value'] = \
                time_label.split('M')[1]
        data['Year of service_close_date'] = data[
            'Year of service_close_date'].astype(int)
        data['Month Value'] = data['Month Value'].astype(int)
        if data['Total GE Cost'].dtype == np.object:
            data['Total GE Cost'] = data['Total GE Cost'].str.replace(',', '') \
                .astype(float)
        else:
            data['Total GE Cost'] = data['Total GE Cost'].astype(float)
    elif metric == 'CSO Open':
        modality_metric_column = 'SII Modality or CSO Modality'
        a_level_modality_column = 'a_level_modality'
        sub_modality_metric_column = 'SUBMODALITY'
        config_modality_column = 'Modality Group (SFDC)'
        #print(data.columns)
        time_labels = data['Year of CASECREATEDDT'].unique()
        for time_label in time_labels:
            data.loc[data['Year of CASECREATEDDT'] == time_label, "Year"] = int(time_label)
            #data.loc[data['Snapshot FW'] == time_label, 'Week'] = int(time_label.split('-')[1])
        data['Year'] = data['Year'].astype(int)
        #data['Week'] = data['Week'].astype(int)
    data[modality_metric_column].fillna('', inplace=True)
    data[sub_modality_metric_column].fillna('', inplace=True)
    #data['Scorecard Product'].fillna('', inplace=True)
    data[a_level_modality_column].fillna('', inplace=True)
    #Change 1
    data['lookup'] = data[modality_metric_column].map(str) + data[sub_modality_metric_column].map(str) #+ data['Scorecard Product'].map(str)
    lookup_for_null_modality = data.loc[data[a_level_modality_column] == '']['lookup'].unique()
    for lookup_value in lookup_for_null_modality:
        lookup_data = modality_lookup.loc[modality_lookup[config_modality_column] == lookup_value]
        if lookup_data.size > 0:
            data.loc[data['lookup'] == lookup_value, 'SC Business Segment'] = lookup_data['Business'].values[0].strip()
            data.loc[data['lookup'] == lookup_value, 'SC Modality'] = lookup_data['Modality'].values[0].strip()
            data.loc[data['lookup'] == lookup_value, 'SC Product Segment'] = 'Other'
        else:
            data.loc[data['lookup'] == lookup_value, 'SC Business Segment'] = 'GEHC Other'
            data.loc[data['lookup'] == lookup_value, 'SC Modality'] = 'Other'
            data.loc[data['lookup'] == lookup_value, 'SC Product SegYment'] = 'Other'
    # update scorecard mapping for records that have value in a_level_modality_code
    lookup_for_not_null_modality = data.loc[data[a_level_modality_column] != '']['lookup'].unique()
    for lookup_value in lookup_for_not_null_modality:
        #print(lookup_value)
        sii_data = sii_hierarchy.loc[sii_hierarchy['lookup value'] == lookup_value]
        if sii_data.size > 0:
            data.loc[data['lookup'] == lookup_value, 'SC Business Segment'] = sii_data['Business'].values[0]
            data.loc[data['lookup'] == lookup_value, 'SC Modality'] = sii_data['Modality'].values[0]
            data.loc[data['lookup'] == lookup_value, 'SC Product Segment'] = sii_data['Product Segment'].values[0]
            data.loc[data['lookup'] == lookup_value, 'SC Product'] = sii_data['sub_family_code (Ultrasound Only) ' \
                                                                              'or product_group_code'].values[0]
        else:
            #print('in exception for lookup value ' + lookup_value)
            a_level_modality_data = data.loc[data['lookup'] == lookup_value][a_level_modality_column]
            if a_level_modality_data.size > 0:
                print(a_level_modality_data.size)
                a_level_modality = a_level_modality_data.unique()
                for modality in a_level_modality:
                    #print(modality)
                    sii_data = sii_hierarchy.loc[sii_hierarchy['a_level_modality_code'] == modality]
                    #print(sii_data.size)
                    if sii_data.size > 0:
                        data.loc[(data['lookup'] == lookup_value) & (data[a_level_modality_column] == modality),
                                 'SC Business Segment'] = sii_data['Business'].values[0].strip()
                        data.loc[(data['lookup'] == lookup_value) & (data[a_level_modality_column] == modality),
                                 'SC Modality'] = sii_data['Modality'].values[0].strip()
                        data.loc[(data['lookup'] == lookup_value) & (data[a_level_modality_column] == modality),
                                 'SC Product Segment'] = sii_data['Product Segment'].values[0].strip()
                        data.loc[(data['lookup'] == lookup_value) & (data[a_level_modality_column] == modality),
                                 'SC Product'] = sii_data['sub_family_code (Ultrasound Only) ' \
                                                          'or product_group_code'] \
                            .values[0].strip()
                    else:
                        data.loc[(data['lookup'] == lookup_value) & (data[a_level_modality_column] == modality),
                                 'SC Business Segment'] = 'GEHC Other'
                        data.loc[(data['lookup'] == lookup_value) & (data[a_level_modality_column] == modality),
                                 'SC Modality'] = 'Other'
                        data.loc[(data['lookup'] == lookup_value) & (data[a_level_modality_column] == modality),
                                 'SC Product Segment'] = 'Other'
            else:
                data.loc[data['lookup'] == lookup_value, 'SC Business Segment'] = 'GEHC Other'
                data.loc[data['lookup'] == lookup_value, 'SC Modality'] = 'Other'
                data.loc[data['lookup'] == lookup_value, 'SC Product Segment'] = 'Other'
    # set scorecard product level data if it is not set yet
#     data['SC Product'].fillna('', inplace=True)
#     blank_scorecard_products = data.loc[data['SC Product'] == '']
#     if blank_scorecard_products.size > 0:
#         products = data.loc[data['SC Product'] == '']['Scorecard Product'].unique()
#         for product in products:
#             scorecard_product = ''
#             if product == '':
#                 scorecard_product = '<blank in source>'
#             else:
#                 scorecard_product = product
#             print(product)
#             data.loc[(data['SC Product'] == '') & (data['Scorecard Product'] == product), 'SC Product'] = \
#                 scorecard_product
    return data

def setup_sii_sc_hierarchy_ib(data, sii_hierarchy):
    data['Modality_Code'].fillna('', inplace=True)
    data['Business_ segment_code'].fillna('', inplace=True)
    data['Sub Family Code (or) Product group code'].fillna('', inplace=True)
    data['lookup'] = data['Modality_Code'].map(str) + data['Business_ segment_code'].map(str) + data[
        'Sub Family Code (or) Product group code'].map(str)
    lookup_for_not_null_modality = data.loc[data['a_level_modality_code'] != '']['lookup'].unique()
    for lookup_value in lookup_for_not_null_modality:
        print(lookup_value)
        sii_data = sii_hierarchy.loc[sii_hierarchy['lookup value'] == lookup_value]
        if sii_data.size > 0:
            data.loc[data['lookup'] == lookup_value, 'SC Business Segment'] = sii_data['Business'].values[0]
            data.loc[data['lookup'] == lookup_value, 'SC Modality'] = sii_data['Modality'].values[0]
            data.loc[data['lookup'] == lookup_value, 'SC Product Segment'] = sii_data['Product Segment'].values[0]
            data.loc[data['lookup'] == lookup_value, 'SC Product'] = sii_data['sub_family_code (Ultrasound Only) ' \
                                                                              'or product_group_code'].values[0]
        else:
            #print('in exception for lookup value ' + lookup_value)
            a_level_modality_data = data.loc[data['lookup'] == lookup_value]['a_level_modality_code']
            if a_level_modality_data.size > 0:
                print(a_level_modality_data.size)
                a_level_modality = a_level_modality_data.unique()
                for modality in a_level_modality:
                    print(modality)
                    sii_data = sii_hierarchy.loc[sii_hierarchy['a_level_modality_code'] == modality]
                    print(sii_data.size)
                    if sii_data.size > 0:
                        data.loc[(data['lookup'] == lookup_value) & (data['a_level_modality_code'] == modality),
                                 'SC Business Segment'] = sii_data['Business'].values[0].strip()
                        data.loc[(data['lookup'] == lookup_value) & (data['a_level_modality_code'] == modality),
                                 'SC Modality'] = sii_data['Modality'].values[0].strip()
                        data.loc[(data['lookup'] == lookup_value) & (data['a_level_modality_code'] == modality),
                                 'SC Product Segment'] = sii_data['Product Segment'].values[0].strip()
                        data.loc[(data['lookup'] == lookup_value) & (data['a_level_modality_code'] == modality),
                                 'SC Product'] = sii_data['sub_family_code (Ultrasound Only) ' \
                                                          'or product_group_code'] \
                            .values[0].strip()
                    else:
                        data.loc[(data['lookup'] == lookup_value) & (data['a_level_modality_code'] == modality),
                                 'SC Business Segment'] = 'GEHC Other'
                        data.loc[(data['lookup'] == lookup_value) & (data['a_level_modality_code'] == modality),
                                 'SC Modality'] = 'Other'
                        data.loc[(data['lookup'] == lookup_value) & (data['a_level_modality_code'] == modality),
                                 'SC Product Segment'] = 'Other'
            else:
                data.loc[data['lookup'] == lookup_value, 'SC Business Segment'] = 'GEHC Other'
                data.loc[data['lookup'] == lookup_value, 'SC Modality'] = 'Other'
                data.loc[data['lookup'] == lookup_value, 'SC Product Segment'] = 'Other'
    # set scorecard product level data if it is not set yet
    data['Sub Family Code (or) Product group code'].fillna('', inplace=True)
    blank_scorecard_products = data.loc[data['Sub Family Code (or) Product group code'] == '']
    if blank_scorecard_products.size > 0:
        products = data.loc[data['SC Product'] == '']['Sub Family Code (or) Product group code'].unique()
        for product in products:
            scorecard_product = ''
            if product == '':
                scorecard_product = '<blank in source>'
            else:
                scorecard_product = product
            print(product)
            data.loc[(data['SC Product'] == '') & (
            data['Sub Family Code (or) Product group code'] == product), 'SC Product'] = \
                scorecard_product
    current_year = datetime.datetime.now().year
    current_month = datetime.datetime.now().month
    data = data.loc[(data['Year'] < current_year) |
                    ((data['Year'] == current_year) &
                     (data['Month'] <= (current_month - 1)))]
    data['lookup'] = data['Modality_Code'].map(str) + data['Business_ segment_code'].map(str) + data[
        'Sub Family Code (or) Product group code'].map(str)
    blank_rows = data.loc[data['lookup'] == '']
    if len(blank_rows) > 0:
        data.loc[data['lookup'] == '', 'SC Business Segment'] = 'GEHC Other'
        data.loc[data['lookup'] == '', 'SC Modality'] = 'Other'
        data.loc[data['lookup'] == '', 'SC Product Segment'] = 'Other'
        products = data.loc[data['lookup'] == '']['Sub Family Code (or) Product group code'].unique()
        for product in products:
            data.loc[(data['lookup'] == '') & (
            data['Sub Family Code (or) Product group code'] == product), 'SC Product'] = '<blank in source>'

    return data

def calculate_ib(score_card, data):
    current_year = datetime.datetime.now().year
    current_month = datetime.datetime.now().month
    for index, row in score_card.iterrows():
        if row['Product Quality Scorecard'] == 'GEHC':
            overall_data = data
            score_card.loc[index, '2016 Total'] = len(overall_data.loc[overall_data['Year'] <= 2016])
            score_card.loc[index, '2017 Total'] = len(overall_data.loc[overall_data['Year'] <= 2017])
            score_card.loc[index, '2018 YTD'] = len(overall_data.loc[(overall_data['Year'] <= 2018)])
            score_card.loc[index, '2017 YTD'] = len(overall_data.loc[(overall_data['Year'] < 2017) |
                                                                      ((overall_data['Year'] == 2017)
                                                                & (overall_data['Month'] <= (current_month - 1)))]) 
 

        elif row['Business'] == 'All' and row['Modality'] == 'All' and row['Product Segment']=='All' \
            and row['Product Quality Scorecard' != 'GEHC']:
            business_data = data.loc[(data['SC Business Segment'] == row['Product Quality Scorecard'])]
            score_card.loc[index, '2016 Total'] = len(business_data.loc[business_data['Year'] <= 2016])
            score_card.loc[index, '2017 Total'] = len(business_data.loc[business_data['Year'] <= 2017])
            score_card.loc[index, '2018 YTD'] = len(business_data.loc[(business_data['Year'] <= 2018)])
            score_card.loc[index, '2017 YTD'] = len(business_data.loc[(business_data['Year'] < 2017) |
                                                                      ((business_data['Year'] == 2017)
                                                                & (business_data['Month'] <= (current_month - 1)))])
        elif row['Business'] != 'All' and row['Modality'] == 'All' and row['Product Segment']=='All' :
            modality_data = data.loc[(data['SC Business Segment'] == row['Business'])
                    & (data['SC Modality'] == row['Product Quality Scorecard'])]
            score_card.loc[index, '2016 Total'] = len(modality_data.loc[modality_data['Year'] <= 2016])
            score_card.loc[index, '2017 Total'] = len(modality_data.loc[modality_data['Year'] <= 2017])
            score_card.loc[index, '2018 YTD'] = len(modality_data.loc[(modality_data['Year'] <= 2018)])
            score_card.loc[index, '2017 YTD'] = len(modality_data.loc[(modality_data['Year'] < 2017) |
                                                                      ((modality_data['Year'] == 2017)
                                                                & (modality_data['Month'] <= (current_month - 1)))])
        elif row['Business'] != 'All' and row['Modality'] != 'All' and row['Product Segment']=='All' :
            ps_data = data.loc[(data['SC Business Segment'] == row['Business'])
                            & (data['SC Modality'] == row['Modality'])
                            & (data['SC Product Segment'] == row['Product Quality Scorecard'])]
            score_card.loc[index, '2016 Total'] = len(ps_data.loc[ps_data['Year'] <= 2016])
            score_card.loc[index, '2017 Total'] = len(ps_data.loc[ps_data['Year'] <= 2017])
            score_card.loc[index, '2018 YTD'] = len(ps_data.loc[(ps_data['Year'] <= 2018)])
            score_card.loc[index, '2017 YTD'] = len(ps_data.loc[(ps_data['Year'] < 2017) |
                                                                      ((ps_data['Year'] == 2017)
                                                                & (ps_data['Month'] <= (current_month - 1)))])
#         elif row['Business'] != 'All' and row['Modality'] != 'All' and row['Product Segment']!='All' :
#             prod_data = data.loc[(data['SC Business Segment'] == row['Business'])
#                             & (data['SC Modality'] == row['Modality'])
#                             & (data['SC Product Segment'] == row['Product Segment'])
#                             & (data['SC Product'] == row['Product Quality Scorecard'])]
#             score_card.loc[index, '2016 Total'] = len(prod_data.loc[prod_data['Year'] <= 2016])
#             score_card.loc[index, '2017 Total'] = len(prod_data.loc[prod_data['Year'] <= 2017])
#             score_card.loc[index, '2018 YTD'] = len(prod_data.loc[(prod_data['Year'] <= 2018)])
#             score_card.loc[index, '2017 YTD'] = len(prod_data.loc[(prod_data['Year'] < 2017) |
#                                                                       ((prod_data['Year'] == 2017)
#                                                                 & (prod_data['Month'] <= (current_month - 1)))])

    return score_card



def calculate_otd(score_card, data):
    current_year = datetime.datetime.now().year
    current_month = datetime.datetime.now().month
    for index, row in score_card.iterrows():
        if row['Product Quality Scorecard'] == 'GEHC':
            gehc_on_time = data[['Year', 'DV On Time?', 'Number of Records']].loc[
                data['DV On Time?'] == 'On Time'].groupby('Year')['Number of Records'].sum()
            gehc_overall = data[['Year', 'DV On Time?', 'Number of Records']] \
                .groupby('Year')['Number of Records'].sum()
            gehc_otd = gehc_on_time / gehc_overall * 100
            for value in gehc_otd.index.get_level_values(0):
                if int(value) != current_year:
                    score_card.loc[index, str(value) + ' Total'] = gehc_otd[value]
                else:
                    score_card.loc[index, str(value) + ' YTD'] = gehc_otd[value]
                    score_card.loc[index, str(value) + ' YTD Installs'] = gehc_overall[value]

            gehc_on_time_ytd = data[['Year', 'Month', 'DV On Time?', 'Number of Records']].loc[
                (data['Year'] == current_year - 1) & (data['Month'] <= current_month - 1)
                & (data['DV On Time?'] == 'On Time')].groupby('Year')['Number of Records'].sum()
            gehc_overall_ytd = data[['Year', 'Month', 'DV On Time?', 'Number of Records']].loc[
                (data['Year'] == current_year - 1) & (data['Month'] <= current_month - 1)] \
                .groupby('Year')['Number of Records'].sum()
            gehc_otd_ytd = gehc_on_time_ytd / gehc_overall_ytd * 100

            try:
                score_card.loc[index, str(gehc_otd_ytd.index.get_level_values(0)[0]) + ' YTD'] = \
                    gehc_otd_ytd[gehc_otd_ytd.index.get_level_values(0)[0]]
                score_card.loc[index, str(gehc_overall_ytd.index.get_level_values(0)[0]) + ' YTD Installs'] = \
                    gehc_overall_ytd[gehc_overall_ytd.index.get_level_values(0)[0]]
            except IndexError:
                score_card.loc[index, str(current_year - 1) + ' YTD'] = ''
                score_card.loc[index, str(current_year - 1) + ' YTD Installs'] = ''
                
        elif row['Product Quality Scorecard'] != 'GEHC' and row['Business'] == 'All' and row['Modality'] == 'All':
            business = data.loc[data['SC Business Segment'] == row['Product Quality Scorecard']]
            #print(row['Product Quality Scorecard'])
            business_on_time = business[['Year', 'DV On Time?', 'Number of Records']].loc[
                business['DV On Time?'] == 'On Time'].groupby('Year')['Number of Records'].sum()
            business_overall = business[['Year', 'DV On Time?', 'Number of Records']] \
                .groupby('Year')['Number of Records'].sum()
            business_otd = business_on_time / business_overall * 100
            for value in business_otd.index.get_level_values(0):
                if int(value) != current_year:
                    score_card.loc[index, str(value) + ' Total'] = business_otd[value]
                else:
                    score_card.loc[index, str(value) + ' YTD'] = business_otd[value]
                    score_card.loc[index, str(value) + ' YTD Installs'] = business_overall[value]

            business_on_time_ytd = business[['Year', 'DV On Time?', 'Number of Records', 'Month']].loc[
                (business['Year'] == current_year - 1) & (business['Month'] <= current_month - 1)
                & (business['DV On Time?'] == 'On Time')].groupby('Year')['Number of Records'].sum()

            business_overall_ytd = business[['Year', 'DV On Time?', 'Number of Records', 'Month']].loc[
                (business['Year'] == current_year - 1) & (business['Month'] <= current_month - 1)] \
                .groupby('Year')['Number of Records'].sum()

            business_otd_ytd = business_on_time_ytd / business_overall_ytd * 100

            try:
                score_card.loc[index, str(business_otd_ytd.index.get_level_values(0)[0]) + ' YTD'] = \
                    business_otd_ytd[business_otd_ytd.index.get_level_values(0)[0]]
                score_card.loc[index, str(business_overall_ytd.index.get_level_values(0)[0]) + ' YTD Installs'] = \
                    business_overall_ytd[business_overall_ytd.index.get_level_values(0)[0]]

            except IndexError:
                score_card.loc[index, str(current_year - 1) + ' YTD'] = ''
                score_card.loc[index, str(current_year - 1) + ' YTD Installs'] = ''

        elif row['Business'] != 'All' and row['Modality'] == 'All':
            modality = data[['SC Business Segment', 'SC Modality', 'Month', 'Year', 'DV On Time?',
                             'Number of Records']].loc[
                (data['SC Business Segment'] == row['Business']) & (\
                    data['SC Modality'] == row['Product Quality Scorecard'])]

            modality_on_time = modality[['Year', 'DV On Time?', 'Number of Records']].loc[\
                modality['DV On Time?'] == 'On Time'].groupby('Year')['Number of Records'].sum()

            modality_overall = modality[['Year', 'DV On Time?', 'Number of Records']]. \
                groupby('Year')['Number of Records'].sum()

            modality_otd = modality_on_time / modality_overall * 100

            for value in modality_otd.index.get_level_values(0):
                if int(value) != current_year:
                    score_card.loc[index, str(value) + ' Total'] = modality_otd[value]
                else:
                    score_card.loc[index, str(value) + ' YTD'] = modality_otd[value]
                    score_card.loc[index, str(value) + ' YTD Installs'] = modality_overall[value]

            modality_on_time_ytd = modality[['Year', 'Month', 'DV On Time?', 'Number of Records']].loc[
                (modality['Year'] == current_year - 1) & (modality['Month'] <= current_month - 1)
                & (modality['DV On Time?'] == 'On Time')].groupby('Year')['Number of Records'].sum()

            modality_overall_ytd = modality[['Year', 'Month', 'DV On Time?', 'Number of Records']].loc[
                (modality['Year'] == current_year - 1) & (modality['Month'] <= current_month - 1)] \
                .groupby('Year')['Number of Records'].sum()

            modality_otd_ytd = modality_on_time_ytd / modality_overall_ytd * 100
            try:
                score_card.loc[index, str(modality_otd_ytd.index.get_level_values(0)[0]) + ' YTD'] = \
                    modality_otd_ytd[modality_otd_ytd.index.get_level_values(0)[0]]
                score_card.loc[index, str(modality_overall_ytd.index.get_level_values(0)[0]) + ' YTD Installs'] = \
                    modality_overall_ytd[modality_overall_ytd.index.get_level_values(0)[0]]
            except IndexError:
                score_card.loc[index, str(current_year - 1) + ' YTD'] = ''
                score_card.loc[index, str(current_year - 1) + ' YTD Installs'] = ''
        elif row['Business'] != 'All' and row['Modality'] != 'All' and row['Product Segment'] == 'All':
            prod_seg = \
                data[['SC Business Segment', 'SC Modality','SC Product Segment','Year', 'DV On Time?', 'Number of Records', 'Month']].loc[\
                    (data['SC Business Segment'] == row['Business']) & (data['SC Modality'] ==
                                                                        row['Modality']) & (\
                        data['SC Product Segment'] == row['Product Quality Scorecard'])]

            prod_seg_on_time = prod_seg[['Year', 'DV On Time?', 'Number of Records']].loc[
                prod_seg['DV On Time?'] == 'On Time'].groupby('Year')['Number of Records'].sum()

            prod_seg_overall = prod_seg[['Year', 'DV On Time?', 'Number of Records']]. \
                groupby('Year')['Number of Records'].sum()

            prod_seg_otd = prod_seg_on_time / prod_seg_overall * 100

            for value in prod_seg_otd.index.get_level_values(0):
                if int(value) != current_year:
                    score_card.loc[index, str(value) + ' Total'] = prod_seg_otd[value]
                else:
                    score_card.loc[index, str(value) + ' YTD'] = prod_seg_otd[value]
                    score_card.loc[index, str(value) + ' YTD Installs'] = prod_seg_overall[value]

            prod_seg_on_time_ytd = prod_seg[['Year', 'Month', 'DV On Time?', 'Number of Records']].loc[
                (prod_seg['Year'] == current_year - 1) & (prod_seg['Month'] <= current_month - 1)
                & (prod_seg['DV On Time?'] == 'On Time')].groupby('Year')['Number of Records'].sum()

            prod_seg_overall_ytd = prod_seg[['Year', 'Month', 'DV On Time?', 'Number of Records']].loc[
                (prod_seg['Year'] == current_year - 1) & (prod_seg['Month'] <= current_month - 1)] \
                .groupby('Year')['Number of Records'].sum()

            prod_seg_otd = prod_seg_on_time_ytd / prod_seg_overall_ytd * 100

            try:
                score_card.loc[index, str(prod_seg_otd.index.get_level_values(0)[0]) + ' YTD'] = \
                    prod_seg_otd[prod_seg_otd.index.get_level_values(0)[0]]
                score_card.loc[index, str(prod_seg_overall_ytd.index.get_level_values(0)[0]) + ' YTD Installs'] = \
                    prod_seg_overall_ytd[prod_seg_overall_ytd.index.get_level_values(0)[0]]
            except IndexError:
                score_card.loc[index, str(current_year - 1) + ' YTD'] = ''
                score_card.loc[index, str(current_year - 1) + ' YTD Installs'] = ''

#         else:
#             product = \
#                 data[['SC Business Segment', 'SC Modality', 'SC Product Segment', 'SC Product',
#                       'Year', 'DV On Time?', 'Number of Records', 'Month']].loc[
#                     (data['SC Business Segment'] == row['Business']) & (data['SC Modality'] ==
#                                                                         row['Modality']) & (
#                         data['SC Product Segment'] == row['Product Segment']) &
#                     (data['SC Product'] == row['Product Quality Scorecard'])]

#             product_on_time = product[['Year', 'DV On Time?', 'Number of Records']].loc[
#                 product['DV On Time?'] == 'On Time'].groupby('Year')['Number of Records'].sum()

#             product_overall = product[['Year', 'DV On Time?', 'Number of Records']]. \
#                 groupby('Year')['Number of Records'].sum()

#             product_otd = product_on_time / product_overall * 100

#             for value in product_otd.index.get_level_values(0):
#                 if int(value) != current_year:
#                     score_card.loc[index, str(value) + ' Total'] = product_otd[value]
#                 else:
#                     score_card.loc[index, str(value) + ' YTD'] = product_otd[value]
#                     score_card.loc[index, str(value) + ' YTD Installs'] = product_overall[value]

#             product_on_time_ytd = product[['Year', 'Month', 'DV On Time?', 'Number of Records']].loc[
#                 (product['Year'] == current_year - 1) & (product['Month'] <= current_month - 1)
#                 & (product['DV On Time?'] == 'On Time')].groupby('Year')['Number of Records'].sum()

#             product_overall_ytd = product[['Year', 'Month', 'DV On Time?', 'Number of Records']].loc[
#                 (product['Year'] == current_year - 1) & (product['Month'] <= current_month - 1)] \
#                 .groupby('Year')['Number of Records'].sum()

#             product_otd_ytd = product_on_time_ytd / product_overall_ytd * 100

#             try:
#                 score_card.loc[index, str(product_otd_ytd.index.get_level_values(0)[0]) + ' YTD'] = \
#                     product_otd_ytd[product_otd_ytd.index.get_level_values(0)[0]]
#                 score_card.loc[index, str(product_overall_ytd.index.get_level_values(0)[0]) + ' YTD Installs'] = \
#                     product_overall_ytd[product_overall_ytd.index.get_level_values(0)[0]]
#             except IndexError:
#                 score_card.loc[index, str(current_year - 1) + ' YTD'] = ''
#                 score_card.loc[index, str(current_year - 1) + ' YTD Installs'] = ''
    # score_card.drop(columns=['Business', 'Modality', 'Product Segment'], inplace=True)
    return score_card


def calculate_oti(score_card, data):
    current_year = datetime.datetime.now().year
    current_month = datetime.datetime.now().month
    for index, row in score_card.iterrows():
        if row['Product Quality Scorecard'] == 'GEHC':
            gehc_on_time = data[['Year', 'Measure Names', 'Measure Values']].loc[
                data['Measure Names'] == '# of On Time Install'].groupby('Year')['Measure Values'].sum()
            gehc_overall = data[['Year', 'Measure Names', 'Measure Values']] \
                .groupby('Year')['Measure Values'].sum()
            gehc_oti = gehc_on_time / gehc_overall * 100
            for value in gehc_oti.index.get_level_values(0):
                if int(value) != current_year:
                    score_card.loc[index, str(value) + ' Total'] = gehc_oti[value]
                else:
                    score_card.loc[index, str(value) + ' YTD'] = gehc_oti[value]
                    score_card.loc[index, str(value) + ' YTD Installs'] = gehc_overall[value]

            gehc_on_time_ytd = data[['Year', 'Month', 'Measure Names', 'Measure Values']].loc[
                (data['Year'] == current_year - 1) & (data['Month'] <= current_month - 1)
                & (data['Measure Names'] == '# of On Time Install')].groupby('Year')['Measure Values'].sum()
            gehc_overall_ytd = data[['Year', 'Month', 'Measure Names', 'Measure Values']].loc[
                (data['Year'] == current_year - 1) & (data['Month'] <= current_month - 1)] \
                .groupby('Year')['Measure Values'].sum()
            gehc_oti_ytd = gehc_on_time_ytd / gehc_overall_ytd * 100

            try:
                score_card.loc[index, str(gehc_oti_ytd.index.get_level_values(0)[0]) + ' YTD'] = \
                    gehc_oti_ytd[gehc_oti_ytd.index.get_level_values(0)[0]]
                score_card.loc[index, str(gehc_overall_ytd.index.get_level_values(0)[0]) + ' YTD Installs'] = \
                    gehc_overall_ytd[gehc_overall_ytd.index.get_level_values(0)[0]]
            except IndexError:
                score_card.loc[index, str(current_year - 1) + ' YTD'] = ''
                score_card.loc[index, str(current_year - 1) + ' YTD Installs'] = ''
                
        elif row['Product Quality Scorecard'] != 'GEHC' and row['Business'] == 'All' and row['Modality'] == 'All':
            business = data.loc[data['SC Business Segment'] == row['Product Quality Scorecard']]
            #print(row['Product Quality Scorecard'])
            business_on_time = business[['Year', 'Measure Names', 'Measure Values']].loc[
                business['Measure Names'] == '# of On Time Install'].groupby('Year')['Measure Values'].sum()
            business_overall = business[['Year', 'Measure Names', 'Measure Values']] \
                .groupby('Year')['Measure Values'].sum()
            business_oti = business_on_time / business_overall * 100
            for value in business_oti.index.get_level_values(0):
                if int(value) != current_year:
                    score_card.loc[index, str(value) + ' Total'] = business_oti[value]
                else:
                    score_card.loc[index, str(value) + ' YTD'] = business_oti[value]
                    score_card.loc[index, str(value) + ' YTD Installs'] = business_overall[value]

            business_on_time_ytd = business[['Year', 'Measure Names', 'Measure Values', 'Month']].loc[
                (business['Year'] == current_year - 1) & (business['Month'] <= current_month - 1)
                & (business['Measure Names'] == '# of On Time Install')].groupby('Year')['Measure Values'].sum()

            business_overall_ytd = business[['Year', 'Measure Names', 'Measure Values', 'Month']].loc[
                (business['Year'] == current_year - 1) & (business['Month'] <= current_month - 1)] \
                .groupby('Year')['Measure Values'].sum()

            business_oti_ytd = business_on_time_ytd / business_overall_ytd * 100

            try:
                score_card.loc[index, str(business_oti_ytd.index.get_level_values(0)[0]) + ' YTD'] = \
                    business_oti_ytd[business_oti_ytd.index.get_level_values(0)[0]]
                score_card.loc[index, str(business_overall_ytd.index.get_level_values(0)[0]) + ' YTD Installs'] = \
                    business_overall_ytd[business_overall_ytd.index.get_level_values(0)[0]]

            except IndexError:
                score_card.loc[index, str(current_year - 1) + ' YTD'] = ''
                score_card.loc[index, str(current_year - 1) + ' YTD Installs'] = ''

        elif row['Business'] != 'All' and row['Modality'] == 'All':
            modality = data[['SC Business Segment', 'SC Modality', 'Month', 'Year', 'Measure Names',
                             'Measure Values']].loc[
                (data['SC Business Segment'] == row['Business']) & (
                    data['SC Modality'] == row['Product Quality Scorecard'])]

            modality_on_time = modality[['Year', 'Measure Names', 'Measure Values']].loc[
                modality['Measure Names'] == '# of On Time Install'].groupby('Year')['Measure Values'].sum()

            modality_overall = modality[['Year', 'Measure Names', 'Measure Values']]. \
                groupby('Year')['Measure Values'].sum()

            modality_oti = modality_on_time / modality_overall * 100

            for value in modality_oti.index.get_level_values(0):
                if int(value) != current_year:
                    score_card.loc[index, str(value) + ' Total'] = modality_oti[value]
                else:
                    score_card.loc[index, str(value) + ' YTD'] = modality_oti[value]
                    score_card.loc[index, str(value) + ' YTD Installs'] = modality_overall[value]

            modality_on_time_ytd = modality[['Year', 'Month', 'Measure Names', 'Measure Values']].loc[
                (modality['Year'] == current_year - 1) & (modality['Month'] <= current_month - 1)
                & (modality['Measure Names'] == '# of On Time Install')].groupby('Year')['Measure Values'].sum()

            modality_overall_ytd = modality[['Year', 'Month', 'Measure Names', 'Measure Values']].loc[
                (modality['Year'] == current_year - 1) & (modality['Month'] <= current_month - 1)] \
                .groupby('Year')['Measure Values'].sum()

            modality_oti_ytd = modality_on_time_ytd / modality_overall_ytd * 100

            try:
                score_card.loc[index, str(modality_oti_ytd.index.get_level_values(0)[0]) + ' YTD'] = \
                    modality_oti_ytd[modality_oti_ytd.index.get_level_values(0)[0]]
                score_card.loc[index, str(modality_overall_ytd.index.get_level_values(0)[0]) + ' YTD Installs'] = \
                    modality_overall_ytd[modality_overall_ytd.index.get_level_values(0)[0]]
            except IndexError:
                score_card.loc[index, str(current_year - 1) + ' YTD'] = ''
                score_card.loc[index, str(current_year - 1) + ' YTD Installs'] = ''

        elif row['Business'] != 'All' and row['Modality'] != 'All' and row['Product Segment'] == 'All':
            prod_seg = \
                data[['SC Business Segment', 'SC Modality', 'SC Product Segment',
                      'Year', 'Measure Names', 'Measure Values', 'Month']].loc[
                    (data['SC Business Segment'] == row['Business']) & (data['SC Modality'] ==
                                                                        row['Modality']) & (
                        data['SC Product Segment'] == row['Product Quality Scorecard'])]

            prod_seg_on_time = prod_seg[['Year', 'Measure Names', 'Measure Values']].loc[
                prod_seg['Measure Names'] == '# of On Time Install'].groupby('Year')['Measure Values'].sum()

            prod_seg_overall = prod_seg[['Year', 'Measure Names', 'Measure Values']]. \
                groupby('Year')['Measure Values'].sum()

            prod_seg_oti = prod_seg_on_time / prod_seg_overall * 100

            for value in prod_seg_oti.index.get_level_values(0):
                if int(value) != current_year:
                    score_card.loc[index, str(value) + ' Total'] = prod_seg_oti[value]
                else:
                    score_card.loc[index, str(value) + ' YTD'] = prod_seg_oti[value]
                    score_card.loc[index, str(value) + ' YTD Installs'] = prod_seg_overall[value]

            prod_seg_on_time_ytd = prod_seg[['Year', 'Month', 'Measure Names', 'Measure Values']].loc[
                (prod_seg['Year'] == current_year - 1) & (prod_seg['Month'] <= current_month - 1)
                & (prod_seg['Measure Names'] == '# of On Time Install')].groupby('Year')['Measure Values'].sum()

            prod_seg_overall_ytd = prod_seg[['Year', 'Month', 'Measure Names', 'Measure Values']].loc[
                (prod_seg['Year'] == current_year - 1) & (prod_seg['Month'] <= current_month - 1)] \
                .groupby('Year')['Measure Values'].sum()

            prod_seg_oti = prod_seg_on_time_ytd / prod_seg_overall_ytd * 100

            try:
                score_card.loc[index, str(prod_seg_oti.index.get_level_values(0)[0]) + ' YTD'] = \
                    prod_seg_oti[prod_seg_oti.index.get_level_values(0)[0]]
                score_card.loc[index, str(prod_seg_overall_ytd.index.get_level_values(0)[0]) + ' YTD Installs'] = \
                    prod_seg_overall_ytd[prod_seg_overall_ytd.index.get_level_values(0)[0]]
            except IndexError:
                score_card.loc[index, str(current_year - 1) + ' YTD'] = ''
                score_card.loc[index, str(current_year - 1) + ' YTD Installs'] = ''

#         else:
#             product = \
#                 data[['SC Business Segment', 'SC Modality', 'SC Product Segment', 'SC Product',
#                       'Year', 'Measure Names', 'Measure Values', 'Month']].loc[
#                     (data['SC Business Segment'] == row['Business']) & (data['SC Modality'] ==
#                                                                         row['Modality']) & (
#                         data['SC Product Segment'] == row['Product Segment']) &
#                     (data['SC Product'] == row['Product Quality Scorecard'])]

#             product_on_time = product[['Year', 'Measure Names', 'Measure Values']].loc[
#                 product['Measure Names'] == '# of On Time Install'].groupby('Year')['Measure Values'].sum()

#             product_overall = product[['Year', 'Measure Names', 'Measure Values']]. \
#                 groupby('Year')['Measure Values'].sum()

#             product_oti = product_on_time / product_overall * 100

#             for value in product_oti.index.get_level_values(0):
#                 if int(value) != current_year:
#                     score_card.loc[index, str(value) + ' Total'] = product_oti[value]
#                 else:
#                     score_card.loc[index, str(value) + ' YTD'] = product_oti[value]
#                     score_card.loc[index, str(value) + ' YTD Installs'] = product_overall[value]

#             product_on_time_ytd = product[['Year', 'Month', 'Measure Names', 'Measure Values']].loc[
#                 (product['Year'] == current_year - 1) & (product['Month'] <= current_month - 1)
#                 & (product['Measure Names'] == '# of On Time Install')].groupby('Year')['Measure Values'].sum()

#             product_overall_ytd = product[['Year', 'Month', 'Measure Names', 'Measure Values']].loc[
#                 (product['Year'] == current_year - 1) & (product['Month'] <= current_month - 1)] \
#                 .groupby('Year')['Measure Values'].sum()

#             product_oti_ytd = product_on_time_ytd / product_overall_ytd * 100

#             try:
#                 score_card.loc[index, str(product_oti_ytd.index.get_level_values(0)[0]) + ' YTD'] = \
#                     product_oti_ytd[product_oti_ytd.index.get_level_values(0)[0]]
#                 score_card.loc[index, str(product_overall_ytd.index.get_level_values(0)[0]) + ' YTD Installs'] = \
#                     product_overall_ytd[product_overall_ytd.index.get_level_values(0)[0]]
#             except IndexError:
#                 score_card.loc[index, str(current_year - 1) + ' YTD'] = ''
#                 score_card.loc[index, str(current_year - 1) + ' YTD Installs'] = ''
    # score_card.drop(columns=['Business', 'Modality', 'Product Segment'], inplace=True)
    return score_card




def calculate_cso_for_customer(score_card, data):
    current_year = datetime.datetime.now().year
    current_month = datetime.datetime.now().month
    for index, row in score_card.iterrows():
        if row['Product Quality Scorecard'] == 'GEHC':
            years = data['Year'].unique()
            for year in years:
                year = int(year)
                max_week_lookup = data.loc[data["Year"] == year]
                max_week_value = max_week_lookup.loc[max_week_lookup['Week'].idxmax()]['Week']
                #print(max_week_value)
                #print(type(max_week_value))
                #print(year)
                red_cso_count = data[['Year', 'Week', 'Measure Values']].loc[(data['Year'] == year)
                                                                      & (data['Week'] == max_week_value)] \
                    .groupby('Year')['Measure Values'].sum()
                for value in red_cso_count.index.get_level_values(0):
                    if int(value) != current_year:
                        score_card.loc[index, str(value) + ' Total'] = red_cso_count[value]
                    else:
                        score_card.loc[index, str(value) + ' YTD'] = red_cso_count[value]
                        previous_year_data = data[['Year', 'Week', 'Measure Values']].loc[(data['Year'] == current_year - 1) &
                                                                                   (data['Week'] == max_week_value)]
                        if (previous_year_data.size > 0):
                            previous_year_ytd = previous_year_data.groupby('Year')['Measure Values'].sum()
                            for value in previous_year_ytd.index.get_level_values(0):
                                score_card.loc[index, str(current_year - 1) + ' YTD'] = \
                                    previous_year_ytd[value]
        elif row['Product Quality Scorecard'] != 'GEHC' and row['Business'] == 'All' and row['Modality'] == 'All':
            business = data.loc[data['SC Business Segment'] == row['Product Quality Scorecard']]
            years = business['Year'].unique()
            for year in years:
                year = int(year)
                max_week_lookup = business.loc[business["Year"] == year]
                max_week_value = max_week_lookup.loc[max_week_lookup['Week'].idxmax()]['Week']
#                 print(max_week_value)
#                 print(type(max_week_value))
#                 print(year)
                red_cso_count = business[['Year', 'Week', 'Measure Values']].loc[(business['Year'] == year)
                                                                          & (business['Week'] == max_week_value)] \
                    .groupby('Year')['Measure Values'].sum()
                for value in red_cso_count.index.get_level_values(0):
                    if int(value) != current_year:
                        score_card.loc[index, str(value) + ' Total'] = red_cso_count[value]
                    else:
                        score_card.loc[index, str(value) + ' YTD'] = red_cso_count[value]
                        previous_year_data = business[['Year', 'Week', 'Measure Values']].loc[
                            (business['Year'] == current_year - 1) &
                            (business['Week'] == max_week_value)]
                        if (previous_year_data.size > 0):
                            previous_year_ytd = previous_year_data.groupby('Year')['Measure Values'].sum()
                            for value in previous_year_ytd.index.get_level_values(0):
                                score_card.loc[index, str(current_year - 1) + ' YTD'] = \
                                    previous_year_ytd[value]
        elif row['Business'] != 'All' and row['Modality'] == 'All':
            modality = data[['SC Business Segment', 'SC Modality', 'Week', 'Year', 'Measure Values']].loc[
                (data['SC Business Segment'] == row['Business']) & (
                    data['SC Modality'] == row['Product Quality Scorecard'])]
            years = modality['Year'].unique()
            for year in years:
                year = int(year)
                max_week_lookup = modality.loc[modality["Year"] == year]
                max_week_value = max_week_lookup.loc[max_week_lookup['Week'].idxmax()]['Week']
#                 print(max_week_value)
#                 print(type(max_week_value))
#                 print(year)
                red_cso_count = modality[['Year', 'Week', 'Measure Values']].loc[(modality['Year'] == year)
                                                                          & (modality['Week'] == max_week_value)] \
                    .groupby('Year')['Measure Values'].sum()
                for value in red_cso_count.index.get_level_values(0):
                    if int(value) != current_year:
                        score_card.loc[index, str(value) + ' Total'] = red_cso_count[value]
                    else:
                        score_card.loc[index, str(value) + ' YTD'] = red_cso_count[value]
                        previous_year_data = modality[['Year', 'Week', 'Measure Values']].loc[
                            (modality['Year'] == current_year - 1) &
                            (modality['Week'] == max_week_value)]
                        if (previous_year_data.size > 0):
                            previous_year_ytd = previous_year_data.groupby('Year')['Measure Values'].sum()
                            for value in previous_year_ytd.index.get_level_values(0):
                                score_card.loc[index, str(current_year - 1) + ' YTD'] = \
                                    previous_year_ytd[value]
        elif row['Business'] != 'All' and row['Modality'] != 'All' and row['Product Segment'] == 'All':
            prod_seg = \
                data[['SC Business Segment', 'SC Modality', 'SC Product Segment',
                      'Year', 'Measure Values', 'Week']].loc[
                    (data['SC Business Segment'] == row['Business']) & (data['SC Modality'] ==
                                                                        row['Modality']) & (
                        data['SC Product Segment'] == row['Product Quality Scorecard'])]
            years = prod_seg['Year'].unique()
            for year in years:
                year = int(year)
                max_week_lookup = prod_seg.loc[prod_seg["Year"] == year]
                max_week_value = max_week_lookup.loc[max_week_lookup['Week'].idxmax()]['Week']
#                 print(max_week_value)
#                 print(type(max_week_value))
#                 print(year)
                red_cso_count = prod_seg[['Year', 'Week', 'Measure Values']].loc[(prod_seg['Year'] == year)
                                                                          & (prod_seg['Week'] == max_week_value)] \
                    .groupby('Year')['Measure Values'].sum()
                for value in red_cso_count.index.get_level_values(0):
                    if int(value) != current_year:
                        score_card.loc[index, str(value) + ' Total'] = red_cso_count[value]
                    else:
                        score_card.loc[index, str(value) + ' YTD'] = red_cso_count[value]
                        previous_year_data = prod_seg[['Year', 'Week', 'Measure Values']].loc[
                            (prod_seg['Year'] == current_year - 1) &
                            (prod_seg['Week'] == max_week_value)]
                        if (previous_year_data.size > 0):
                            previous_year_ytd = previous_year_data.groupby('Year')['Measure Values'].sum()
                            for value in previous_year_ytd.index.get_level_values(0):
                                score_card.loc[index, str(current_year - 1) + ' YTD'] = \
                                    previous_year_ytd[value]
#         else:
#             product = \
#                 data[['SC Business Segment', 'SC Modality', 'SC Product Segment', 'SC Product',
#                       'Year', 'Measure Values', 'Week']].loc[
#                     (data['SC Business Segment'] == row['Business']) & (data['SC Modality'] ==
#                                                                         row['Modality']) & (
#                         data['SC Product Segment'] == row['Product Segment']) &
#                     (data['SC Product'] == row['Product Quality Scorecard'])]
#             years = product['Year'].unique()
#             for year in years:
#                 year = int(year)
#                 max_week_lookup = product.loc[product["Year"] == year]
#                 max_week_value = max_week_lookup.loc[max_week_lookup['Week'].idxmax()]['Week']
# #                 print(max_week_value)
# #                 print(type(max_week_value))
# #                 print(year)
#                 red_cso_count = product[['Year', 'Week', 'Measure Values']].loc[(product['Year'] == year)
#                                                                          & (product['Week'] == max_week_value)] \
#                     .groupby('Year')['Measure Values'].sum()
#                 for value in red_cso_count.index.get_level_values(0):
#                     if int(value) != current_year:
#                         score_card.loc[index, str(value) + ' Total'] = red_cso_count[value]
#                     else:
#                         score_card.loc[index, str(value) + ' YTD'] = red_cso_count[value]
#                         previous_year_data = product[['Year', 'Week', 'Measure Values']].loc[
#                             (product['Year'] == current_year - 1) &
#                             (product['Week'] == max_week_value)]
#                         if (previous_year_data.size > 0):
#                             previous_year_ytd = previous_year_data.groupby('Year')['Measure Values'].sum()
#                             for value in previous_year_ytd.index.get_level_values(0):
#                                 score_card.loc[index, str(current_year - 1) + ' YTD'] = \
#                                     previous_year_ytd[value]
    # score_card.drop(columns=['Business', 'Modality', 'Product Segment'], inplace=True)
    return score_card

def calculate_cso_open_for_customer(score_card, data):
    current_year = datetime.datetime.now().year
    #current_month = datetime.datetime.now().month
    for index, row in score_card.iterrows():
        if row['Product Quality Scorecard'] == 'GEHC':
            years = data['Year'].unique()
            for year in years:
                year = int(year)
                red_cso_count = data[['Year', 'Number of Records']].loc[(data['Year'] == year)] \
                    .groupby('Year')['Number of Records'].sum()
                for value in red_cso_count.index.get_level_values(0):
                    if int(value) != current_year:
                        score_card.loc[index, str(value) + ' Total'] = red_cso_count[value]
                    else:
                        score_card.loc[index, str(value) + ' YTD'] = red_cso_count[value]
                        previous_year_data = data[['Year', 'Number of Records']].loc[(data['Year'] == current_year - 1)]
                        if (previous_year_data.size > 0):
                            previous_year_ytd = previous_year_data.groupby('Year')['Number of Records'].sum()
                            for value in previous_year_ytd.index.get_level_values(0):
                                score_card.loc[index, str(current_year - 1) + ' YTD'] = \
                                    previous_year_ytd[value]
        elif row['Product Quality Scorecard'] != 'GEHC' and row['Business'] == 'All' and row['Modality'] == 'All':
            business = data.loc[data['SC Business Segment'] == row['Product Quality Scorecard']]
            years = business['Year'].unique()
            for year in years:
                year = int(year)
                red_cso_count = business[['Year', 'Number of Records']].loc[(business['Year'] == year)] \
                    .groupby('Year')['Number of Records'].sum()
                for value in red_cso_count.index.get_level_values(0):
                    if int(value) != current_year:
                        score_card.loc[index, str(value) + ' Total'] = red_cso_count[value]
                    else:
                        score_card.loc[index, str(value) + ' YTD'] = red_cso_count[value]
                        previous_year_data = business[['Year', 'Number of Records']].loc[
                            (business['Year'] == current_year - 1)]
                        if (previous_year_data.size > 0):
                            previous_year_ytd = previous_year_data.groupby('Year')['Number of Records'].sum()
                            for value in previous_year_ytd.index.get_level_values(0):
                                score_card.loc[index, str(current_year - 1) + ' YTD'] = \
                                    previous_year_ytd[value]
        elif row['Business'] != 'All' and row['Modality'] == 'All':
            modality = data[['SC Business Segment', 'SC Modality',  'Year', 'Number of Records']].loc[
                (data['SC Business Segment'] == row['Business']) & (
                    data['SC Modality'] == row['Product Quality Scorecard'])]
            years = modality['Year'].unique()
            for year in years:
                year = int(year)
                red_cso_count = modality[['Year', 'Number of Records']].loc[(modality['Year'] == year)] \
                    .groupby('Year')['Number of Records'].sum()
                for value in red_cso_count.index.get_level_values(0):
                    if int(value) != current_year:
                        score_card.loc[index, str(value) + ' Total'] = red_cso_count[value]
                    else:
                        score_card.loc[index, str(value) + ' YTD'] = red_cso_count[value]
                        previous_year_data = modality[['Year', 'Number of Records']].loc[
                            (modality['Year'] == current_year - 1)]
                        if (previous_year_data.size > 0):
                            previous_year_ytd = previous_year_data.groupby('Year')['Number of Records'].sum()
                            for value in previous_year_ytd.index.get_level_values(0):
                                score_card.loc[index, str(current_year - 1) + ' YTD'] = \
                                    previous_year_ytd[value]
        elif row['Business'] != 'All' and row['Modality'] != 'All' and row['Product Segment'] == 'All':
            prod_seg = \
                data[['SC Business Segment', 'SC Modality', 'SC Product Segment',
                      'Year', 'Number of Records']].loc[
                    (data['SC Business Segment'] == row['Business']) & (data['SC Modality'] ==
                                                                        row['Modality']) & (
                        data['SC Product Segment'] == row['Product Quality Scorecard'])]
            years = prod_seg['Year'].unique()
            for year in years:
                year = int(year)
                red_cso_count = prod_seg[['Year', 'Number of Records']].loc[(prod_seg['Year'] == year)] \
                    .groupby('Year')['Number of Records'].sum()
                for value in red_cso_count.index.get_level_values(0):
                    if int(value) != current_year:
                        score_card.loc[index, str(value) + ' Total'] = red_cso_count[value]
                    else:
                        score_card.loc[index, str(value) + ' YTD'] = red_cso_count[value]
                        previous_year_data = prod_seg[['Year', 'Number of Records']].loc[
                            (prod_seg['Year'] == current_year - 1) ]
                        if (previous_year_data.size > 0):
                            previous_year_ytd = previous_year_data.groupby('Year')['Number of Records'].sum()
                            for value in previous_year_ytd.index.get_level_values(0):
                                score_card.loc[index, str(current_year - 1) + ' YTD'] = \
                                    previous_year_ytd[value]
#         else:
#             product = \
#                 data[['SC Business Segment', 'SC Modality', 'SC Product Segment', 'SC Product',
#                       'Year', 'Number of Records']].loc[
#                     (data['SC Business Segment'] == row['Business']) & (data['SC Modality'] ==
#                                                                         row['Modality']) & (
#                         data['SC Product Segment'] == row['Product Segment']) &
#                     (data['SC Product'] == row['Product Quality Scorecard'])]
#             years = product['Year'].unique()
#             for year in years:
#                 year = int(year)
#                 red_cso_count = product[['Year', 'Number of Records']].loc[(product['Year'] == year)] \
#                     .groupby('Year')['Number of Records'].sum()
#                 for value in red_cso_count.index.get_level_values(0):
#                     if int(value) != current_year:
#                         score_card.loc[index, str(value) + ' Total'] = red_cso_count[value]
#                     else:
#                         score_card.loc[index, str(value) + ' YTD'] = red_cso_count[value]
#                         previous_year_data = product[['Year', 'Number of Records']].loc[
#                             (product['Year'] == current_year - 1) ]
#                         if (previous_year_data.size > 0):
#                             previous_year_ytd = previous_year_data.groupby('Year')['Number of Records'].sum()
#                             for value in previous_year_ytd.index.get_level_values(0):
#                                 score_card.loc[index, str(current_year - 1) + ' YTD'] = \
#                                     previous_year_ytd[value]
    # score_card.drop(columns=['Business', 'Modality', 'Product Segment'], inplace=True)
    return score_card

def setup_chu_hierarchy(data, chu_hierarachy):
    data['Modality'].fillna('', inplace=True)
    data['Product Group / Modality Segment - IB'].fillna('', inplace=True)
    #print(data['Product Group / Modality Segment - IB'].apply(lambda x: print(x)))
    data['lookup'] = data['Modality'].map(str) + data['Product Group / Modality Segment - IB'].map(str)
    time_labels = data['By Time Label'].unique()
    for time_label in time_labels:
        data.loc[data['By Time Label'] == time_label, "Year"] = time_label.split('M')[0]
        data.loc[data['By Time Label'] == time_label, "Month"] = time_label.split('M')[1]
    data['Year'] = data['Year'].astype(int)
    data['Month'] = data['Month'].astype(int)
    modalities = data['Modality'].unique()
    for modality in modalities:
        chu_modality_data = chu_hierarachy.loc[chu_hierarachy['Modality'] == modality]
        if chu_modality_data.size > 0:
            data.loc[data['Modality'] == modality, 'SC Business Segment'] = chu_modality_data['Business for Scorecard'] \
                .values[0]
            data.loc[data['Modality'] == modality, 'SC Modality'] = chu_modality_data['Modality for Scorecard'] \
                .values[0]
            prod_segments = data.loc[data['Modality'] == modality]['lookup'].unique()
            for prod_segment in prod_segments:
                prod_segment_data = chu_hierarachy.loc[chu_hierarachy['Lookup Value'] == prod_segment]
                if prod_segment_data.size > 0:
                    data.loc[(data['Modality'] == modality) & (data['lookup'] == prod_segment), 'SC Product Segment'] = \
                        prod_segment_data['Product Segment for Scorecard'].values[0]
                    data.loc[(data['Modality'] == modality) & (data['lookup'] == prod_segment), 'SC Product'] = \
                        prod_segment_data['Modality Segment / Product Group'].values[0]
                else:
                    data.loc[(data['Modality'] == modality) & (data['lookup'] == prod_segment), 'SC Product Segment'] = \
                        'Other'
        else:
            data.loc[data['Modality'] == modality, 'SC Business Segment'] = 'GEHC Other'
            data.loc[data['Modality'] == modality, 'SC Modality'] = 'Other'
            data.loc[data['Modality'] == modality, 'SC Product Segment'] = 'Other'

    data['SC Product'].fillna('', inplace=True)
    blank_scorecard_products = data.loc[data['SC Product'] == '']
    if blank_scorecard_products.size > 0:
        products = data.loc[data['SC Product'] == '']['Product Group / Modality Segment - IB'].unique()
        for product in products:
            scorecard_product = ''
            if product == '':
                scorecard_product = '<blank in source>'
            else:
                scorecard_product = product
            data.loc[(data['SC Product'] == '') & (data['Product Group / Modality Segment - IB'] == product),
                     'SC Product'] = scorecard_product
    return data

def setup_chu_hierarchy_ib(data, chu_hierarachy):
    data['Modality_Code'].fillna('', inplace=True)
    data['Sub Family Code (or) Product group code'].fillna('', inplace=True)
    #print(data['Product Group / Modality Segment - IB'].apply(lambda x: print(x)))
    data['lookup'] = data['Modality_Code'].map(str) + data['Sub Family Code (or) Product group code'].map(str)
   
    modalities = data['Modality_Code'].unique()
    for modality in modalities:
        chu_modality_data = chu_hierarachy.loc[chu_hierarachy['Modality'] == modality]
        if chu_modality_data.size > 0:
            data.loc[data['Modality_Code'] == modality, 'SC Business Segment'] = chu_modality_data['Business for Scorecard'] \
                .values[0]
            data.loc[data['Modality_Code'] == modality, 'SC Modality'] = chu_modality_data['Modality for Scorecard'] \
                .values[0]
            prod_segments = data.loc[data['Modality_Code'] == modality]['lookup'].unique()
            for prod_segment in prod_segments:
                prod_segment_data = chu_hierarachy.loc[chu_hierarachy['Lookup Value'] == prod_segment]
                if prod_segment_data.size > 0:
                    data.loc[(data['Modality_Code'] == modality) & (data['lookup'] == prod_segment), 'SC Product Segment'] = \
                        prod_segment_data['Product Segment for Scorecard'].values[0]
                    data.loc[(data['Modality_Code'] == modality) & (data['lookup'] == prod_segment), 'SC Product'] = \
                        prod_segment_data['Modality Segment / Product Group'].values[0]
                else:
                    data.loc[(data['Modality_Code'] == modality) & (data['lookup'] == prod_segment), 'SC Product Segment'] = \
                        'Other'
        else:
            data.loc[data['Modality_Code'] == modality, 'SC Business Segment'] = 'GEHC Other'
            data.loc[data['Modality_Code'] == modality, 'SC Modality'] = 'Other'
            data.loc[data['Modality_Code'] == modality, 'SC Product Segment'] = 'Other'

    data['SC Product'].fillna('', inplace=True)
    blank_scorecard_products = data.loc[data['SC Product'] == '']
    if blank_scorecard_products.size > 0:
        products = data.loc[data['SC Product'] == '']['Sub Family Code (or) Product group code'].unique()
        for product in products:
            scorecard_product = ''
            if product == '':
                scorecard_product = '<blank in source>'
            else:
                scorecard_product = product
            data.loc[(data['SC Product'] == '') & (data['Sub Family Code (or) Product group code'] == product),
                     'SC Product'] = scorecard_product
    return data


def setup_ifr_hierarchy(ifr, sii, ifr_modalities):
    ifr['Measure Values'] = ifr['Measure Values'].astype(int)
    time_labels = ifr['Time Label'].unique()
    for time_label in time_labels:
        ifr.loc[ifr['Time Label'] == time_label, 'Month'] = time_label[-2:]
        ifr.loc[ifr['Time Label'] == time_label, 'Year'] = time_label[:4]
    ifr['Month'] = ifr['Month'].astype(int)
    ifr['Year'] = ifr['Year'].astype(int)
    ifr['Modality_Code'].fillna('', inplace=True)
    ifr['Business_ segment_code'].fillna('', inplace=True)
    ifr['Sub Family Code (or) Product group code'].fillna('', inplace=True)
    ifr['lookup'] = ifr['Modality_Code'].map(str) + ifr['Business_ segment_code'].map(str) + ifr['Sub Family Code (or) Product group code'].map(str)
    lookup_values = ifr['lookup'].unique()
    for lookup_value in lookup_values:
        #print(lookup_value)
        sii_data = sii.loc[sii['lookup value'] == lookup_value]
        #print(sii_data.size)
        if sii_data.size > 0:
            ifr.loc[ifr['lookup'] == lookup_value, 'SC Business Segment'] = sii_data['Business'].values[0]
            ifr.loc[ifr['lookup'] == lookup_value, 'SC Modality'] = sii_data['Modality'].values[0]
            ifr.loc[ifr['lookup'] == lookup_value, 'SC Product Segment'] = sii_data['Product Segment'].values[0]
            ifr.loc[ifr['lookup'] == lookup_value, 'SC Product'] = sii_data['sub_family_code (Ultrasound Only) ' \
                                                                            'or product_group_code'].values[0]
        else:
            ifr_modality_values = ifr.loc[ifr['lookup'] == lookup_value]['Modality_Code'].unique()
            for ifr_modality_value in ifr_modality_values:
                modality_data = ifr_modalities.loc[ifr_modalities['SII'] == ifr_modality_value]
                if modality_data.size > 0:
                    ifr.loc[(ifr['lookup'] == lookup_value) & (ifr['Modality_Code'] == ifr_modality_value),
                            'SC Business Segment'] = modality_data['Business']
                    ifr.loc[(ifr['lookup'] == lookup_value) & (ifr['Modality_Code'] == ifr_modality_value),
                            'SC Modality'] = modality_data['PQ Modality']
                    ifr.loc[(ifr['lookup'] == lookup_value) & (ifr['Modality_Code'] == ifr_modality_value),
                            'SC Product Segment'] = 'Other'
                    ifr.loc[(ifr['lookup'] == lookup_value) & (ifr['Modality_Code'] == ifr_modality_value),
                            'SC Product'] = ifr.loc[(ifr['lookup'] == lookup_value) & (ifr['Modality_Code'] ==
                                    ifr_modality_value)]['Sub Family Code (or) Product group code']
    ifr['SC Business Segment'].fillna('', inplace=True)
    ifr['SC Modality'].fillna('', inplace=True)
    ifr['SC Product Segment'].fillna('', inplace=True)
    ifr['SC Product'].fillna('', inplace=True)
    null_segment_data = ifr.loc[ifr['SC Business Segment'] == '']
    if null_segment_data.size > 0:
        ifr.loc[ifr['SC Business Segment'] == '', 'SC Product Segment'] = 'Other'
        ifr.loc[ifr['SC Business Segment'] == '', 'SC Modality'] = 'Other'
        ifr.loc[ifr['SC Business Segment'] == '', 'SC Business Segment'] = 'GEHC Other'
    null_modality = ifr.loc[ifr['SC Modality'] == '']
    if null_modality.size > 0:
        ifr.loc[ifr['SC Modality'] == '', 'SC Modality'] = 'Other'
    null_prod_seg = ifr.loc[ifr['SC Product Segment'] == '']
    if null_prod_seg.size > 0:
        ifr.loc[ifr['SC Product Segment'] == '', 'SC Product Segment'] = 'Other'
    null_products = ifr.loc[ifr['SC Product'] == '']
    if null_products.size>0:
        products = ifr.loc[ifr['SC Product']=='']['Sub Family Code (or) Product group code'].unique()
        for product in products:
            if product == '':
                product = '<blank in source>'
            ifr.loc[(ifr['SC Product'] == '') &(ifr['Sub Family Code (or) Product group code'] == product),
                                    'SC Product'] = product

    return ifr


def calculate_ifr_rate(score_card, data):
    current_year = datetime.datetime.now().year
    current_month = datetime.datetime.now().month
    for index, row in score_card.iterrows():
        #print(row['Product Quality Scorecard'])
        if row['Product Quality Scorecard'] == 'GEHC':
            gehc_sr = data[['Year', 'Measure Names', 'Measure Values']].loc[\
                (data['Measure Names'] == 'SR Count')].groupby('Year')['Measure Values'].sum()

            gehc_ib = (data[['Year', 'Measure Names', 'Measure Values']].loc[
                           (data['Measure Names'] == 'IB Count')].groupby('Year')[
                           'Measure Values'].sum())

            ifr_rate = gehc_sr / gehc_ib
            #print(ifr_rate)

            for value in ifr_rate.index.get_level_values(0):
                if int(value) != current_year:
                    score_card.loc[index, str(value) + ' Total'] = ifr_rate[value]
                else:
                    score_card.loc[index, str(value) + ' YTD'] = ifr_rate[value]
                    score_card.loc[index, str(value) + ' YTD Installs'] = gehc_ib[value]

            gehc_sr_ytd = data[['Year', 'Month', 'Measure Names', 'Measure Values']].loc[
                (data['Year'] == current_year - 1) & (data['Month'] <= current_month - 1)
                & (data['Measure Names'] == 'SR Count')].groupby('Year')['Measure Values'].sum()

            gehc_ib_ytd = (data[['Year', 'Month', 'Measure Names', 'Measure Values']].loc[
                               (data['Year'] == current_year - 1) &
                               (data['Month'] <= current_month - 1) &
                               (data['Measure Names'] == 'IB Count')].groupby('Year')[
                               'Measure Values'].sum())

            ifr_rate_ytd = gehc_sr_ytd / gehc_ib_ytd
            try:
                score_card.loc[index, str(ifr_rate_ytd.index.get_level_values(0)[0]) + ' YTD'] = \
                    ifr_rate_ytd[ifr_rate_ytd.index.get_level_values(0)[0]]
                score_card.loc[index, str(gehc_ib_ytd.index.get_level_values(0)[0]) + ' YTD Installs'] = \
                    gehc_ib_ytd[gehc_ib_ytd.index.get_level_values(0)[0]]
            except IndexError:
                score_card.loc[index, str(current_year - 1) + ' YTD'] = ''
                score_card.loc[index, str(current_year - 1) + ' YTD Installs'] = ''
                
        elif row['Product Quality Scorecard'] != 'GEHC' and row['Business'] == 'All' and row['Modality'] == 'All' \
                and row['Product Segment'] == 'All':
            business = data[['Year', 'Measure Names', 'Month', 'Measure Values']].loc[(data['SC Business Segment'] ==
                                                                                 row['Product Quality Scorecard'])]
            #print(row['Product Quality Scorecard'])
            sr_business = business[['Year', 'Measure Names', 'Measure Values']].loc[
                business['Measure Names'] == 'SR Count'].groupby('Year')['Measure Values'].sum()
            #print(sr_business)
            ib_business = (business[['Year', 'Measure Names', 'Measure Values']].loc[
                               business['Measure Names'] == 'IB Count'].groupby('Year')[
                               'Measure Values'].sum())
            #print(ib_business)

            ifr_rate_business = sr_business / ib_business
            #print(ifr_rate_business)
            for value in ifr_rate_business.index.get_level_values(0):
                if int(value) != current_year:
                    score_card.loc[index, str(value) + ' Total'] = ifr_rate_business[value]
                else:
                    score_card.loc[index, str(value) + ' YTD'] = ifr_rate_business[value]
                    score_card.loc[index, str(value) + ' YTD Installs'] = ib_business[value]

            business_sr_ytd = business[['Year', 'Month', 'Measure Names', 'Measure Values']].loc[
                (business['Year'] == current_year - 1) & (business['Month'] <= current_month - 1)
                & (business['Measure Names'] == 'SR Count')].groupby('Year')['Measure Values'].sum()

            business_ib_ytd = (business[['Year', 'Month', 'Measure Names', 'Measure Values']].loc[
                                   (business['Year'] == current_year - 1) &
                                   (business['Month'] <= current_month - 1) &
                                   (business['Measure Names'] == 'IB Count')].groupby('Year')[
                                   'Measure Values'].sum())

            business_ifr_rate_ytd = business_sr_ytd / business_ib_ytd
            try:
                score_card.loc[index, str(business_ifr_rate_ytd.index.get_level_values(0)[0]) + ' YTD'] = \
                    business_ifr_rate_ytd[business_ifr_rate_ytd.index.get_level_values(0)[0]]
                score_card.loc[index, str(business_ib_ytd.index.get_level_values(0)[0]) + ' YTD Installs'] = \
                    business_ib_ytd[business_ib_ytd.index.get_level_values(0)[0]]
            except IndexError:
                score_card.loc[index, str(current_year - 1) + ' YTD'] = ''
                score_card.loc[index, str(current_year - 1) + ' YTD Installs'] = ''

        elif row['Business'] != 'All' and row['Modality'] == 'All' and row['Product Segment'] == 'All':
            modality = data[['SC Business Segment', 'SC Modality', 'Month', 'Year', 'Measure Names',
                             'Measure Values']].loc[
                (data['SC Business Segment'] == row['Business']) & ((
                                                                        data['SC Modality'] == row[
                                                                            'Product Quality Scorecard']))]

            sr_modality = modality[['Year', 'Measure Names', 'Measure Values']].loc[
                modality['Measure Names'] == 'SR Count'].groupby('Year')['Measure Values'].sum()

            ib_modality = (modality[['Year', 'Measure Names', 'Measure Values']].loc[
                               modality['Measure Names'] == 'IB Count'].groupby('Year')[
                               'Measure Values'].sum())

            ifr_rate_modality = sr_modality / ib_modality

            for value in ifr_rate_modality.index.get_level_values(0):
                if int(value) != current_year:
                    score_card.loc[index, str(value) + ' Total'] = ifr_rate_modality[value]
                else:
                    score_card.loc[index, str(value) + ' YTD'] = ifr_rate_modality[value]
                    score_card.loc[index, str(value) + ' YTD Installs'] = ib_modality[value]

            modality_sr_ytd = modality[['Year', 'Month', 'Measure Names', 'Measure Values']].loc[
                (modality['Year'] == current_year - 1) & (modality['Month'] <= current_month - 1)
                & (modality['Measure Names'] == 'SR Count')].groupby('Year')['Measure Values'].sum()

            modality_ib_ytd = (modality[['Year', 'Month', 'Measure Names', 'Measure Values']].loc[
                                   (modality['Year'] == current_year - 1) &
                                   (modality['Month'] <= current_month - 1) &
                                   (modality['Measure Names'] == 'IB Count')].groupby('Year')[
                                   'Measure Values'].sum())

            modality_ifr_rate_ytd = modality_sr_ytd / modality_ib_ytd
            try:
                score_card.loc[index, str(modality_ifr_rate_ytd.index.get_level_values(0)[0]) + ' YTD'] = \
                    modality_ifr_rate_ytd[modality_ifr_rate_ytd.index.get_level_values(0)[0]]
                score_card.loc[index, str(modality_ib_ytd.index.get_level_values(0)[0]) + ' YTD Installs'] = \
                    modality_ib_ytd[modality_ib_ytd.index.get_level_values(0)[0]]
            except IndexError:
                score_card.loc[index, str(current_year - 1) + ' YTD'] = ''
                score_card.loc[index, str(current_year - 1) + ' YTD Installs'] = ''

        elif row['Business'] != 'All' and row['Modality'] != 'All' and row['Product Segment'] == 'All':
            prod_seg = \
                data[['SC Business Segment', 'SC Modality', 'SC Product Segment',
                      'Year', 'Month', 'Measure Names', 'Measure Values']].loc[
                    (data['SC Business Segment'] == row['Business']) & (data['SC Modality'] ==
                                                                        row['Modality']) & (
                        data['SC Product Segment'] == row['Product Quality Scorecard'])]

            sr_prod_seg = prod_seg[['Year', 'Month', 'Measure Names', 'Measure Values']].loc[
                prod_seg['Measure Names'] == 'SR Count'].groupby(['Year'])['Measure Values'].sum()

            ib_prod_seg = (prod_seg[['Year', 'Measure Names', 'Measure Values']].loc[
                               prod_seg['Measure Names'] == 'IB Count'].groupby('Year')[
                               'Measure Values'].sum())

            ifr_rate_prod_seg = sr_prod_seg / ib_prod_seg

            for value in ifr_rate_prod_seg.index.get_level_values(0):
                if int(value) != current_year:
                    score_card.loc[index, str(value) + ' Total'] = ifr_rate_prod_seg[value]
                else:
                    score_card.loc[index, str(value) + ' YTD'] = ifr_rate_prod_seg[value]
                    score_card.loc[index, str(value) + ' YTD Installs'] = ib_prod_seg[value]

            prod_seg_sr_ytd = prod_seg[['Year', 'Month', 'Measure Names', 'Measure Values']].loc[
                (prod_seg['Year'] == current_year - 1) & (prod_seg['Month'] <= current_month - 1)
                & (prod_seg['Measure Names'] == 'SR Count')].groupby('Year')['Measure Values'].sum()

            prod_seg_ib_ytd = (prod_seg[['Year', 'Month', 'Measure Names', 'Measure Values']].loc[
                                   (prod_seg['Year'] == current_year - 1) &
                                   (prod_seg['Month'] <= current_month - 1) &
                                   (prod_seg['Measure Names'] == 'IB Count')].groupby('Year')[
                                   'Measure Values'].sum())

            prod_seg_ifr_rate_ytd = prod_seg_sr_ytd / prod_seg_ib_ytd
            try:
                score_card.loc[index, str(prod_seg_ifr_rate_ytd.index.get_level_values(0)[0]) + ' YTD'] = \
                    prod_seg_ifr_rate_ytd[prod_seg_ifr_rate_ytd.index.get_level_values(0)[0]]
                score_card.loc[index, str(prod_seg_ib_ytd.index.get_level_values(0)[0]) + ' YTD Installs'] = \
                    prod_seg_ib_ytd[prod_seg_ib_ytd.index.get_level_values(0)[0]]
            except IndexError:
                score_card.loc[index, str(current_year - 1) + ' YTD'] = ''
                score_card.loc[index, str(current_year - 1) + ' YTD Installs'] = ''

#         else:

#             product = \
#                 data[['SC Business Segment', 'SC Modality', 'SC Product Segment', 'SC Product',
#                       'Year', 'Month', 'Measure Names', 'Measure Values']].loc[
#                     (data['SC Business Segment'] == row['Business']) & (data['SC Modality'] == row['Modality'])
#                     & (data['SC Product Segment'] == row['Product Segment']) &
#                     (data['SC Product'] == row['Product Quality Scorecard']) ]

#             sr_prod = product[['Year', 'Month', 'Measure Names', 'Measure Values']].loc[
#                 product['Measure Names'] == 'SR Count'].groupby(['Year'])['Measure Values'].sum()

#             ib_prod = (product[['Year', 'Measure Names', 'Measure Values']].loc[
#                            product['Measure Names'] == 'IB Count'].groupby('Year')[
#                            'Measure Values'].sum())

#             ifr_rate_prod = sr_prod / ib_prod

#             for value in ifr_rate_prod.index.get_level_values(0):
#                 if int(value) != current_year:
#                     score_card.loc[index, str(value) + ' Total'] = ifr_rate_prod[value]
#                 else:
#                     score_card.loc[index, str(value) + ' YTD'] = ifr_rate_prod[value]
#                     score_card.loc[index, str(value) + ' YTD Installs'] = ib_prod[value]

#             prod_sr_ytd = product[['Year', 'Month', 'Measure Names', 'Measure Values']].loc[
#                 (product['Year'] == current_year - 1) & (product['Month'] <= current_month - 1)
#                 & (product['Measure Names'] == 'SR Count')].groupby('Year')['Measure Values'].sum()

#             prod_ib_ytd = (product[['Year', 'Month', 'Measure Names', 'Measure Values']].loc[
#                                (product['Year'] == current_year - 1) &
#                                (product['Month'] <= current_month - 1) &
#                                (product['Measure Names'] == 'IB Count')].groupby('Year')[
#                                'Measure Values'].sum())

#             prod_ifr_rate_ytd = prod_sr_ytd / prod_ib_ytd
#             try:
#                 score_card.loc[index, str(prod_ifr_rate_ytd.index.get_level_values(0)[0]) + ' YTD'] = \
#                     prod_ifr_rate_ytd[prod_ifr_rate_ytd.index.get_level_values(0)[0]]
#                 score_card.loc[index, str(prod_ib_ytd.index.get_level_values(0)[0]) + ' YTD Installs'] = \
#                     prod_ib_ytd[prod_ib_ytd.index.get_level_values(0)[0]]
#             except IndexError:
#                 score_card.loc[index, str(current_year - 1) + ' YTD'] = ''
#                 score_card.loc[index, str(current_year - 1) + ' YTD Installs'] = ''
    return score_card




#def calculate_cpu_rate(score_card, data):
    
def calculate_cpu_rate(score_card, data):
    current_year = datetime.datetime.now().year
    current_month = datetime.datetime.now().month
    for index, row in score_card.iterrows():
        if row['Product Quality Scorecard'] == 'GEHC':
            gehc_complaint = data[['Year', 'Measure Names', 'Measure Values']].loc[data['Measure Names'] == '# of Complaints'].groupby('Year')['Measure Values'].sum()
            
            gehc_ib_sum = data[['Year','Month', 'Measure Names', 'Measure Values']].loc[data['Measure Names'] == 'Average IB Over Time'].groupby(['Year','Month'])[
                           'Measure Values'].sum().reset_index()
            
            gehc_ib = gehc_ib_sum.groupby('Year')['Measure Values'].sum()

            complaint_rate = gehc_complaint / gehc_ib

            for value in complaint_rate.index.get_level_values(0):
                if int(value) != current_year:
                    score_card.loc[index, str(value) + ' Total'] = complaint_rate[value] * gehc_ib_sum.loc[gehc_ib_sum['Year'] == value]['Month'].nunique()
#                     score_card.loc[index, str(value) + ' Total Complaints'] = gehc_complaint[value]
                else:
                    #print(gehc_ib_sum_cy["Measure Values"].loc[gehc_ib_sum_cy['Year'] == value])
                    score_card.loc[index, str((value))  + ' YTD'] = complaint_rate[value] * gehc_ib_sum.loc[gehc_ib_sum['Year'] == value]['Month'].nunique()
                    score_card.loc[index, str((value))  + ' YTD IB Installs'] = gehc_ib_sum["Measure Values"].loc[gehc_ib_sum['Year'] == value].sum()/(gehc_ib_sum.loc[gehc_ib_sum['Year'] == value]['Month'].nunique())
                    score_card.loc[index, str((value))  + ' YTD IB Complaints'] = gehc_complaint[value]

            gehc_complaint_ytd = data[['Year', 'Measure Names', 'Measure Values']].loc[\
                (data['Year'] == current_year - 1) & (data['Month'] <= current_month - 1)\
                & (data['Measure Names'] == '# of Complaints')].groupby(['Year'])['Measure Values']\
                                                .sum()

            gehc_ib_ytd_sum = data[['Year', 'Month', 'Measure Names', 'Measure Values']].loc[\
                               (data['Year'] == current_year - 1) &\
                               (data['Month'] <= current_month - 1) &\
                               (data['Measure Names'] == 'Average IB Over Time')].groupby(['Year','Month'])[\
                               'Measure Values'].sum().reset_index()
            gehc_ib_ytd = gehc_ib_ytd_sum.groupby('Year')['Measure Values'].sum()/gehc_ib_ytd_sum['Month'].nunique()

            complaint_rate_ytd = gehc_complaint_ytd / gehc_ib_ytd

            try:
                score_card.loc[index, str(int(complaint_rate_ytd.index.get_level_values(0)[0])) + ' YTD'] = \
                    complaint_rate_ytd[complaint_rate_ytd.index.get_level_values(0)[0]]
                score_card.loc[index, str(int(gehc_ib_ytd.index.get_level_values(0)[0])) + ' YTD IB Installs'] = \
                    gehc_ib_ytd[gehc_ib_ytd.index.get_level_values(0)[0]]
                score_card.loc[index, str(int(gehc_complaint_ytd.index.get_level_values(0)[0])) + ' YTD IB Complaints'] = \
                    gehc_complaint_ytd[gehc_complaint_ytd.index.get_level_values(0)[0]]
            except IndexError:
                score_card.loc[index, str(current_year - 1) + ' YTD'] = ''
                score_card.loc[index, str(current_year - 1) + ' YTD IB Installs'] = ''
                score_card.loc[index, str(current_year - 1) + ' YTD IB Complaints'] = ''
        elif row['Product Quality Scorecard'] != 'GEHC' and row['Business'] == 'All' and row['Modality'] == 'All':
            business = data.loc[data['SC Business Segment'] == row['Product Quality Scorecard']]
            complaint_business = business[['Year', 'Measure Names', 'Measure Values']].loc[\
                business['Measure Names'] == '# of Complaints'].groupby('Year')['Measure Values'].sum()
            ib_business_sum = business[['Year','Month','Measure Names', 'Measure Values']].loc[\
                               business['Measure Names'] == 'Average IB Over Time'].groupby(['Year','Month'])[
                               'Measure Values'].sum().reset_index()

            ib_business = ib_business_sum.groupby('Year')['Measure Values'].sum()
            complaint_rate_business = complaint_business / ib_business

            for value in complaint_rate_business.index.get_level_values(0):
                print(complaint_business[value])
                if int(value) != current_year:
                    score_card.loc[index, str(value) + ' Total'] = complaint_rate_business[value]  * ib_business_sum \
                    .loc[ib_business_sum['Year'] == value]['Month'].nunique()
#                     score_card.loc[index, str(value) + ' Total Complaints'] = complaint_business[value] 
                else:
                    score_card.loc[index, str(int(value))  + ' YTD'] = complaint_rate_business[value] * ib_business_sum \
                    .loc[ib_business_sum['Year'] == value]['Month'].nunique()
                    score_card.loc[index, str(int(value))  + ' YTD IB Installs'] = ib_business_sum["Measure Values"].loc[ib_business_sum['Year'] == value].sum()/(ib_business_sum.loc[ib_business_sum['Year'] == value]['Month'].nunique())
                    score_card.loc[index, str(int(value))  + ' YTD IB Complaints'] = complaint_business[value]

            business_complaint_ytd = business[['Year', 'Month', 'Measure Names', 'Measure Values']].loc[\
                (business['Year'] == current_year - 1) & (business['Month'] <= current_month - 1)\
                & (business['Measure Names'] == '# of Complaints')].groupby('Year')['Measure Values'].sum()

            business_ib_ytd_sum = business[['Year', 'Month', 'Measure Names', 'Measure Values']].loc[\
                                   (business['Year'] == current_year - 1) &\
                                   (business['Month'] <= current_month - 1) &\
                                   (business['Measure Names'] == 'Average IB Over Time')].groupby(['Year','Month'])[
                                   'Measure Values'].sum().reset_index()
            business_ib_ytd = business_ib_ytd_sum.groupby('Year')['Measure Values'].sum()/\
                              business_ib_ytd_sum['Month'].nunique()
            business_complaint_rate_ytd = business_complaint_ytd / business_ib_ytd

            try:
                score_card.loc[index, str(int(business_complaint_rate_ytd.index.get_level_values(0)[0])) + ' YTD'] = \
                    business_complaint_rate_ytd[business_complaint_rate_ytd.index.get_level_values(0)[0]]
                score_card.loc[index, str(int(business_ib_ytd.index.get_level_values(0)[0])) + ' YTD IB Installs'] = \
                    business_ib_ytd[business_ib_ytd.index.get_level_values(0)[0]]
                score_card.loc[index, str(int(business_complaint_ytd.index.get_level_values(0)[0])) + ' YTD IB Complaints'] = \
                    business_complaint_ytd[business_complaint_ytd.index.get_level_values(0)[0]]

            except IndexError:
                score_card.loc[index, str(current_year - 1) + ' YTD'] = ''
                score_card.loc[index, str(current_year - 1) + ' YTD IB Installs'] = ''
                score_card.loc[index, str(current_year - 1) + ' YTD IB Complaints'] = ''

        elif row['Business'] != 'All' and row['Modality'] == 'All':
            modality = data[['SC Business Segment', 'SC Modality', 'Month', 'Year', 'Measure Names',
                             'Measure Values']].loc[(data['SC Business Segment'] == row['Business']) & (
                    data['SC Modality'] == row['Product Quality Scorecard'])]

            complaint_modality = modality[['Year', 'Measure Names', 'Measure Values']].loc[\
                modality['Measure Names'] == '# of Complaints'].groupby('Year')['Measure Values'].sum()

            ib_modality_sum = modality[['Year', 'Month','Measure Names', 'Measure Values']].loc[\
                               modality['Measure Names'] == 'Average IB Over Time'].groupby(['Year','Month'])[\
                               'Measure Values'].sum().reset_index()
            ib_modality = ib_modality_sum.groupby('Year')['Measure Values'].sum()

            complaint_rate_modality = complaint_modality / ib_modality

            for value in complaint_rate_modality.index.get_level_values(0):
                if int(value) != current_year:
                    score_card.loc[index, str(value) + ' Total'] = complaint_rate_modality[value] * ib_modality_sum \
                    .loc[ib_modality_sum['Year'] == value]['Month'].nunique()
                else:
                    score_card.loc[index, str(int(value))  + ' YTD'] = complaint_rate_modality[value] * ib_modality_sum \
                    .loc[ib_modality_sum['Year'] == value]['Month'].nunique()
                    score_card.loc[index, str(int(value))  + ' YTD IB Installs'] = ib_modality_sum["Measure Values"].loc[ib_modality_sum['Year'] == value].sum()/(ib_modality_sum.loc[ib_modality_sum['Year'] == value]['Month'].nunique())
                    score_card.loc[index, str(int(value))  + ' YTD IB Complaints'] = complaint_modality[value]
                    
            modality_complaint_ytd = modality[['Year', 'Month', 'Measure Names', 'Measure Values']].loc[\
                (modality['Year'] == current_year - 1) & (modality['Month'] <= current_month - 1)\
                & (modality['Measure Names'] == '# of Complaints')].groupby('Year')['Measure Values'].sum()

            modality_ib_ytd_sum = modality[['Year', 'Month', 'Measure Names', 'Measure Values']].loc[\
                                   (modality['Year'] == current_year - 1) &\
                                   (modality['Month'] <= current_month - 1) &\
                                   (modality['Measure Names'] == 'Average IB Over Time')].groupby(['Year','Month'])[\
                                   'Measure Values'].sum().reset_index()
            modality_ib_ytd = modality_ib_ytd_sum.groupby('Year')['Measure Values'].sum()/modality_ib_ytd_sum['Month']\
                                                                            .nunique()
            modality_complaint_rate_ytd = modality_complaint_ytd / modality_ib_ytd
            try:
                score_card.loc[index, str(int(modality_complaint_rate_ytd.index.get_level_values(0)[0])) + ' YTD'] = \
                    modality_complaint_rate_ytd[modality_complaint_rate_ytd.index.get_level_values(0)[0]]
                score_card.loc[index, str(int(modality_ib_ytd.index.get_level_values(0)[0])) + ' YTD IB Installs'] = \
                    modality_ib_ytd[modality_ib_ytd.index.get_level_values(0)[0]]
                score_card.loc[index, str(int(modality_complaint_ytd.index.get_level_values(0)[0])) + ' YTD IB Complaints'] = \
                    modality_complaint_ytd[modality_complaint_ytd.index.get_level_values(0)[0]]
            except IndexError:
                score_card.loc[index, str(current_year - 1) + ' YTD'] = ''
                score_card.loc[index, str(current_year - 1) + ' YTD IB Installs'] = ''
                score_card.loc[index, str(current_year - 1) + ' YTD IB Complaints'] = ''

        elif row['Business'] != 'All' and row['Modality'] != 'All' and row['Product Segment'] == 'All':
            prod_seg = \
                data[['SC Business Segment', 'SC Modality', 'SC Product Segment',
                      'Year', 'Measure Names', 'Measure Values', 'Month']].loc[(data['SC Business Segment'] == row['Business']) & (data['SC Modality'] ==
                                                                        row['Modality']) & (
                        data['SC Product Segment'] == row['Product Quality Scorecard'])]

            complaint_prod_seg = prod_seg[['Year', 'Measure Names', 'Measure Values']].loc[\
                prod_seg['Measure Names'] == '# of Complaints'].groupby('Year')['Measure Values'].sum()

            ib_prod_seg_sum = prod_seg[['Year','Month', 'Measure Names', 'Measure Values']].loc[\
                               prod_seg['Measure Names'] == 'Average IB Over Time'].groupby(['Year','Month'])[\
                               'Measure Values'].sum().reset_index()
            ib_prod_seg = ib_prod_seg_sum.groupby('Year')['Measure Values'].sum()

            complaint_rate_prod_seg = complaint_prod_seg / ib_prod_seg

            for value in complaint_prod_seg.index.get_level_values(0):
                if int(value) != current_year:
                    score_card.loc[index, str(value) + ' Total'] = complaint_rate_prod_seg[value] * ib_prod_seg_sum \
                    .loc[ib_prod_seg_sum['Year'] == value]['Month'].nunique()
                else:
                    score_card.loc[index, str(int(value))  + ' YTD'] = complaint_rate_prod_seg[value] * ib_prod_seg_sum.loc[ib_prod_seg_sum['Year'] == value]['Month'].nunique()
                    score_card.loc[index, str(int(value))  + ' YTD IB Installs'] = ib_prod_seg_sum["Measure Values"].loc[ib_prod_seg_sum['Year'] == value].sum()/(ib_prod_seg_sum.loc[ib_prod_seg_sum['Year'] == value]['Month'].nunique())
                    score_card.loc[index, str(int(value))  + ' YTD IB Complaints'] = complaint_prod_seg[value]
#                     try:
#                         score_card.loc[index, str(int(value))  + ' YTD'] = complaint_rate_prod_seg[value] * ib_prod_seg_sum.loc[ib_prod_seg_sum['Year'] == value]['Month'].nunique()
#                         score_card.loc[index, str(int(value))  + ' YTD IB Installs'] = ib_prod_seg_sum["Measure Values"].loc[ib_prod_seg_sum['Year'] == value].sum()/(ib_prod_seg_sum.loc[ib_prod_seg_sum['Year'] == value]['Month'].nunique())
#                         score_card.loc[index, str(int(value))  + ' YTD IB Complaints'] = complaint_prod_seg[value]
#                     except IndexError:
#                         score_card.loc[index, str(current_year) + ' YTD'] = ''
#                         score_card.loc[index, str(current_year) + ' YTD IB Installs'] = ''
#                         score_card.loc[index, str(current_year) + ' YTD IB Complaints'] = ''

            prod_seg_complaint_ytd = prod_seg[['Year', 'Month', 'Measure Names', 'Measure Values']].loc[\
                (prod_seg['Year'] == current_year - 1) & (prod_seg['Month'] <= current_month - 1)\
                & (prod_seg['Measure Names'] == '# of Complaints')].groupby('Year')['Measure Values'].sum()

            prod_seg_ib_ytd_sum = prod_seg[['Year', 'Month', 'Measure Names', 'Measure Values']].loc[
                                   (prod_seg['Year'] == current_year - 1) &
                                   (prod_seg['Month'] <= current_month - 1) &
                                   (prod_seg['Measure Names'] == 'Average IB Over Time')].groupby(['Year','Month'])[
                                   'Measure Values'].sum().reset_index()
            prod_seg_ib_ytd = prod_seg_ib_ytd_sum.groupby('Year')['Measure Values'].sum()/prod_seg_ib_ytd_sum['Month']\
                                                                    .nunique()
            prod_seg_complaint_rate_ytd = prod_seg_complaint_ytd / prod_seg_ib_ytd
            try:
                score_card.loc[index, str(int(prod_seg_complaint_rate_ytd.index.get_level_values(0)[0])) + ' YTD'] = \
                    prod_seg_complaint_rate_ytd[prod_seg_complaint_rate_ytd.index.get_level_values(0)[0]]
                score_card.loc[index, str(int(prod_seg_ib_ytd.index.get_level_values(0)[0])) + ' YTD IB Installs'] = \
                    prod_seg_ib_ytd[prod_seg_ib_ytd.index.get_level_values(0)[0]]
                score_card.loc[index, str(int(prod_seg_complaint_ytd.index.get_level_values(0)[0])) + ' YTD IB Complaints'] = \
                    prod_seg_complaint_ytd[prod_seg_complaint_ytd.index.get_level_values(0)[0]]
            except IndexError:
                score_card.loc[index, str(current_year - 1) + ' YTD'] = ''
                score_card.loc[index, str(current_year - 1) + ' YTD IB Installs'] = ''
                score_card.loc[index, str(current_year - 1) + ' YTD IB Complaints'] = ''

#         else:
#             product = \
#                 data[['SC Business Segment', 'SC Modality', 'SC Product Segment', 'SC Product',
#                       'Year', 'Measure Names', 'Measure Values', 'Month']].loc[(data['SC Business Segment'] == row['Business']) & (data['SC Modality'] ==
#                                                                         row['Modality']) & (
#                         data['SC Product Segment'] == row['Product Segment']) &(data['SC Product'] == row['Product Quality Scorecard'])]

#             complaint_prod = product[['Year', 'Measure Names', 'Measure Values']].loc[\
#                 product['Measure Names'] == '# of Complaints'].groupby(['Year'])['Measure Values'].sum()
            
#             ib_prod_sum = product[['Year','Month', 'Measure Names', 'Measure Values']].loc[\
#                                product['Measure Names'] == 'Average IB Over Time'].groupby(['Year','Month'])[\
#                                'Measure Values'].sum().reset_index()
#             ib_prod = ib_prod_sum.groupby('Year')['Measure Values'].sum()

#             complaint_rate_prod = complaint_prod / ib_prod


#             for value in complaint_rate_prod.index.get_level_values(0):
#                 if int(value) != current_year:
#                     score_card.loc[index, str(int(value))  + ' Total'] = complaint_rate_prod[value] * ib_prod_sum \
#                     .loc[ib_prod_sum['Year'] == value]['Month'].nunique()
#                 else:
#                     score_card.loc[index, str(int(value))  + ' YTD'] = complaint_rate_prod[value] * ib_prod_sum.loc[ib_prod_sum['Year'] == value]['Month'].nunique()
#                     score_card.loc[index, str(int(value))  + ' YTD IB Installs'] = ib_prod_sum["Measure Values"].loc[ib_prod_sum['Year'] == value].sum()/(ib_prod_sum.loc[ib_prod_sum['Year'] == value]['Month'].nunique())
#                     score_card.loc[index, str(int(value))  + ' YTD IB Complaints'] = complaint_prod[value]
# #                     try:
# #                         score_card.loc[index, str(int(value))  + ' YTD'] = complaint_rate_prod[value] * ib_prod_sum.loc[ib_prod_sum['Year'] == value]['Month'].nunique()
# #                         score_card.loc[index, str(int(value))  + ' YTD IB Installs'] = ib_prod_sum["Measure Values"].loc[ib_prod_sum['Year'] == value].sum()/(ib_prod_sum.loc[ib_prod_sum['Year'] == value]['Month'].nunique())
# #                         score_card.loc[index, str(int(value))  + ' YTD IB Complaints'] = complaint_prod[value]
# #                     except IndexError:
# #                         score_card.loc[index, str(current_year) + ' YTD'] = ''
# #                         score_card.loc[index, str(current_year) + ' YTD IB Installs'] = ''
# #                         score_card.loc[index, str(current_year) + ' YTD IB Complaints'] = ''
                    
#             prod_complaint_ytd = product[['Year', 'Month', 'Measure Names', 'Measure Values']].loc[\
#                 (product['Year'] == current_year - 1) & (product['Month'] <= current_month - 1)\
#                 & (product['Measure Names'] == '# of Complaints')].groupby('Year')['Measure Values'].sum()
            
#             prod_ib_ytd_sum = product[['Year', 'Month', 'Measure Names', 'Measure Values']].loc[\
#                                    (product['Year'] == current_year - 1) &\
#                                    (product['Month'] <= current_month - 1) &\
#                                    (product['Measure Names'] == 'Average IB Over Time')].groupby(['Year','Month'])[
#                                    'Measure Values'].sum().reset_index()
#             prod_ib_ytd = prod_ib_ytd_sum.groupby('Year')['Measure Values'].sum()/prod_ib_ytd_sum['Month']\
#                                                                     .nunique()
#             prod_complaint_rate_ytd = prod_complaint_ytd / prod_ib_ytd


#             try:
#                 score_card.loc[index, str(int(prod_complaint_rate_ytd.index.get_level_values(0)[0])) + ' YTD'] = \
#                     prod_complaint_rate_ytd[prod_complaint_rate_ytd.index.get_level_values(0)[0]]
#                 score_card.loc[index, str(int(prod_ib_ytd.index.get_level_values(0)[0])) + ' YTD IB Installs'] = \
#                     prod_ib_ytd[prod_ib_ytd.index.get_level_values(0)[0]]
#                 score_card.loc[index, str(int(prod_complaint_ytd.index.get_level_values(0)[0])) + ' YTD IB Complaints'] = \
#                     prod_complaint_ytd[prod_complaint_ytd.index.get_level_values(0)[0]]
#             except IndexError:
#                 score_card.loc[index, str(current_year - 1) + ' YTD'] = ''
#                 score_card.loc[index, str(current_year - 1) + ' YTD IB Installs'] = ''
#                 score_card.loc[index, str(current_year - 1) + ' YTD IB Complaints'] = ''

    return score_card

def setup_obf_hierarchy(data, sii, metric_name):
    if metric_name == 'OBF':
        data.rename(
            columns={'Month': 'month_name',
                     'SII Modality': 'SII Modality Code',
                     'SII Product Group': 'SII Product Group',
                     'SII Sub Modality': 'Sub Modality Code',
                     'Year': 'Year of Process Date',
                     '# of SRs': 'Measure Values'}, inplace=True)
        data['SII Modality Code'].fillna('', inplace=True)
        data['Sub Modality Code'].fillna('', inplace=True)
        data['SII Product Group'].fillna('', inplace=True)
        data['Year of Process Date'] = data['Year of Process Date'].astype(int)
        abbr_to_num = {name: num for num, name in enumerate(calendar.month_name) if num}
        month_names = data['month_name'].unique()
        for month in month_names:
            data.loc[data['month_name'] == month, 'Month'] = abbr_to_num[month]
    data['look_up'] = data['SII Modality Code'].map(str) + data['Sub Modality Code'].map(str) + \
                        data['SII Product Group'].map(str)
    print(data.head)
    lookup_values = data['look_up'].unique()
    for lookup_value in lookup_values:
        sii_data = sii.loc[sii['lookup value'].apply(
            lambda x: x.lower()) == lookup_value.lower()]
        if sii_data.size > 0:
            data.loc[data['look_up'] == lookup_value, 'SC Business Segment'] = \
                sii_data['Business'].values[0]
            data.loc[data['look_up'] == lookup_value, 'SC Modality'] = \
                sii_data['Modality'].values[0]
            data.loc[data['look_up'] == lookup_value, 'SC Product Segment'] = \
                sii_data['Product Segment'].values[0]
            data.loc[data['look_up'] == lookup_value, 'SC Product'] = \
                sii_data['sub_family_code (Ultrasound Only) ' \
                         'or product_group_code'].values[0]
        else:
            data.loc[data[
                         'look_up'] == lookup_value, 'SC Business Segment'] = 'GEHC Other'
            data.loc[data['look_up'] == lookup_value, 'SC Modality'] = 'Other'
            data.loc[data[
                         'look_up'] == lookup_value, 'SC Product Segment'] = 'Other'
            data.loc[data['look_up'] == lookup_value, 'SC Product'] = \
                data.loc[data['look_up'] == lookup_value] \
                    ['SII Product Group']
    # commenting section to consider all data upto date of refresh

    current_year = datetime.datetime.now().year
    current_month = datetime.datetime.now().month
    data = data.loc[(data['Year of Process Date'] < current_year) | (
        (data['Year of Process Date'] == current_year)
        & (data['Month'] <= current_month - 1))]

    data['SC Product'].fillna('', inplace=True)
    blank_scorecard_products = data.loc[data['SC Product'] == '']
    if blank_scorecard_products.size > 0:
        products = data.loc[data['SC Product'] == ''][
            'SII Product Group'].unique()
        for product in products:
            scorecard_product = ''
            if product == '':
                scorecard_product = '<blank in source>'
            else:
                scorecard_product = product

            data.loc[(data['SC Product'] == '') & (
                data['SII Product Group'] == product), 'SC Product'] = \
                scorecard_product
    return data


def calculate_obf(score_card, data):
    current_year = datetime.datetime.now().year
    current_month = datetime.datetime.now().month
    
    for index, row in score_card.iterrows():
        if row['Product Quality Scorecard'] == 'GEHC':
            gehc = data.groupby('Year of Process Date')['Measure Values'].sum()

            for value in gehc.index.get_level_values(0):
                if int(value) != current_year:
                    score_card.loc[index, str(value) + ' Total'] = gehc[value]
                else:
                    score_card.loc[index, str(value) + ' YTD'] = gehc[value]
            gehc_ytd = \
                data[['Measure Values', 'Year of Process Date', 'Month']].loc[
                    (data['Year of Process Date'] == current_year - 1) & (
                        data['Month'] <= current_month - 1)]
            gehc_ytd_group = gehc_ytd.groupby('Year of Process Date')[
                'Measure Values'].sum()
            if gehc_ytd_group.size > 0:
                score_card.loc[index, str(
                    gehc_ytd_group.index.get_level_values(0)[0]) + ' YTD'] = \
                    gehc_ytd_group[gehc_ytd_group.index.get_level_values(0)[0]]

        elif row['Product Quality Scorecard'] != 'GEHC' and row[
            'Business'] == 'All' and row['Modality'] == 'All':
            business = data.loc[data['SC Business Segment'] == row[
                'Product Quality Scorecard']]
            business_group = business.groupby('Year of Process Date')[
                'Measure Values'].sum()
            for value in business_group.index.get_level_values(0):
                if value != current_year:
                    score_card.loc[index, str(value) + ' Total'] = \
                        business_group[value]
                else:
                    score_card.loc[index, str(value) + ' YTD'] = \
                        business_group[value]
            business_ytd = \
                business[
                    ['Measure Values', 'Year of Process Date', 'Month']].loc[
                    (business['Year of Process Date'] == current_year - 1) & (
                        business['Month'] <= current_month - 1)]
            business_ytd_group = business_ytd.groupby('Year of Process Date')[
                'Measure Values'].sum()
            if business_ytd_group.size > 0:
                score_card.loc[index, str(
                    business_ytd_group.index.get_level_values(0)[
                        0]) + ' YTD'] = \
                    business_ytd_group[
                        business_ytd_group.index.get_level_values(0)[0]]
        elif row['Business'] != 'All' and row['Modality'] == 'All' and row[
            'Product Quality Scorecard'] != 'LCS':
            modality = \
                data[['SC Business Segment', 'SC Modality', 'Measure Values',
                      'Year of Process Date', 'Month']].loc[
                    (data['SC Business Segment'] == row['Business']) & (
                        data['SC Modality'] == row[
                            'Product Quality Scorecard'])]
            modality_group = modality.groupby('Year of Process Date')[
                'Measure Values'].sum()
            for value in modality_group.index.get_level_values(0):
                if value != current_year:
                    score_card.loc[index, str(value) + ' Total'] = \
                        modality_group[value]
                else:
                    score_card.loc[index, str(value) + ' YTD'] = \
                        modality_group[value]
            modality_ytd = \
                modality[
                    ['Measure Values', 'Year of Process Date', 'Month']].loc[
                    (modality['Year of Process Date'] == current_year - 1) & (
                        modality['Month'] <= current_month - 1)]
            modality_ytd_group = modality_ytd.groupby('Year of Process Date')[
                'Measure Values'].sum()
            if modality_ytd_group.size > 0:
                score_card.loc[index, str(
                    modality_ytd_group.index.get_level_values(0)[
                        0]) + ' YTD'] = \
                    modality_ytd_group[
                        modality_ytd_group.index.get_level_values(0)[0]]
        elif row['Business'] != 'All' and row['Modality'] == 'All' and row[
            'Product Quality Scorecard'] == 'LCS':
            modality = \
                data[['SC Business Segment', 'SC Modality', 'Measure Values',
                      'Year of Process Date', 'Month']].loc[
                    (data['SC Business Segment'] == row['Business'])
                    & (data['SC Modality'] != 'ULTRASOUND')
                    & (data['SC Modality'] != 'EXCLUDED')
                    & (data['SC Modality'] != 'OTHER')]
            modality_group = modality.groupby('Year of Process Date')[
                'Measure Values'].sum()
            for value in modality_group.index.get_level_values(0):
                if value != current_year:
                    score_card.loc[index, str(value) + ' Total'] = \
                        modality_group[value]
                else:
                    score_card.loc[index, str(value) + ' YTD'] = \
                        modality_group[value]
            modality_ytd = \
                modality[
                    ['Measure Values', 'Year of Process Date', 'Month']].loc[
                    (modality['Year of Process Date'] == current_year - 1) & (
                        modality['Month'] <= current_month - 1)]
            modality_ytd_group = modality_ytd.groupby('Year of Process Date')[
                'Measure Values'].sum()
            if modality_ytd_group.size > 0:
                score_card.loc[index, str(
                    modality_ytd_group.index.get_level_values(0)[
                        0]) + ' YTD'] = \
                    modality_ytd_group[
                        modality_ytd_group.index.get_level_values(0)[0]]
        elif row['Business'] != 'All' and row['Modality'] != 'All' and row[
            'Product Segment'] == 'All':
            prod_seg = \
                data[['SC Business Segment', 'SC Modality',
                      'SC Product Segment', 'Measure Values',
                      'Year of Process Date', 'Month']].loc[
                    (data['SC Business Segment'] == row['Business']) & (
                        data['SC Modality'] == row['Modality']) & (
                        data['SC Product Segment'] == row[
                            'Product Quality Scorecard'])]
            prod_seg_group = prod_seg.groupby('Year of Process Date')[
                'Measure Values'].sum()
            for value in prod_seg_group.index.get_level_values(0):
                if value != current_year:
                    score_card.loc[index, str(value) + ' Total'] = \
                        prod_seg_group[value]
                else:
                    score_card.loc[index, str(value) + ' YTD'] = \
                        prod_seg_group[value]
            try:
                prod_seg_ytd = prod_seg[
                    ['Measure Values', 'Year of Process Date', 'Month']].loc[
                    (prod_seg['Year of Process Date'] == current_year - 1) & (
                        prod_seg['Month'] <= current_month - 1)]

                prod_seg_ytd_group = \
                    prod_seg_ytd.groupby('Year of Process Date')[
                        'Measure Values'].sum()

                if prod_seg_ytd_group.size > 0:
                    score_card.loc[index, str(
                        prod_seg_ytd_group.index.get_level_values(0)[
                            0]) + ' YTD'] = \
                        prod_seg_ytd_group[
                            prod_seg_ytd_group.index.get_level_values(0)[0]]
            except KeyError:
                score_card.loc[index, str(current_year - 1) + ' YTD'] = ''
            except IndexError:
                score_card.loc[index, str(current_year - 1) + ' YTD'] = ''
#         else:
#             product = \
#                 data[['SC Business Segment', 'SC Modality',
#                       'SC Product Segment', 'SC Product', 'Measure Values',
#                       'Year of Process Date', 'Month']].loc[
#                     (data['SC Business Segment'] == row['Business']) & (
#                         data['SC Modality'] == row['Modality']) & (
#                         data['SC Product Segment'] == row['Product Segment'])
#                     & (data['SC Product'] == row['Product Quality Scorecard'])]
#             product_group = product.groupby('Year of Process Date')[
#                 'Measure Values'].sum()
#             for value in product_group.index.get_level_values(0):
#                 if value != current_year:
#                     score_card.loc[index, str(value) + ' Total'] = \
#                         product_group[value]
#                 else:
#                     score_card.loc[index, str(value) + ' YTD'] = product_group[
#                         value]
#             try:
#                 product_ytd = product[
#                     ['Measure Values', 'Year of Process Date', 'Month']].loc[
#                     (product['Year of Process Date'] == current_year - 1) & (
#                         product['Month'] <= current_month - 1)]

#                 product_ytd_group = \
#                     product_ytd.groupby('Year of Process Date')[
#                         'Measure Values'].sum()

#                 if product_ytd_group.size > 0:
#                     score_card.loc[index, str(
#                         product_ytd_group.index.get_level_values(0)[
#                             0]) + ' YTD'] = \
#                         product_ytd_group[
#                             product_ytd_group.index.get_level_values(0)[0]]
#             except KeyError:
#                 score_card.loc[index, str(current_year - 1) + ' YTD'] = ''
#             except IndexError:
#                 score_card.loc[index, str(current_year - 1) + ' YTD'] = ''

    # score_card.drop(columns=['Business', 'Modality'], inplace=True)
    return score_card



def calculate_elf(score_card, data):
    from datetime import date
    current_year = datetime.datetime.now().year
    current_month = datetime.datetime.now().month
    today = datetime.date.today()
    weekday = today.weekday()
    print("WeekDay", weekday)
    start_delta = datetime.timedelta(days=weekday, weeks=1)
    start_of_week = today - start_delta
    print("Delta", start_delta)
    weekNumber = start_of_week.isocalendar()[1]
    print("WeekNumber", weekNumber)
    # weekNumber = date.today().isocalendar()[1]
    # weekNumber = date.today().isocalendar()[1]
    for index, row in score_card.iterrows():

        if row['Product Quality Scorecard'] == 'GEHC':
            gehc = data.groupby('Year of service_close_date')[
                'Total GE Cost'].sum()

            for value in gehc.index.get_level_values(0):
                if int(value) != current_year:
                    score_card.loc[index, str(value) + ' Total'] = gehc[value]
                else:
                    score_card.loc[index, str(value) + ' YTD'] = gehc[value]
            gehc_ytd = data[['Total GE Cost', 'Year of service_close_date',
                             'Week Number']].loc[
                (data['Year of service_close_date'] == current_year - 1) & (
                        data['Week Number'] <= weekNumber)]
            gehc_ytd_group = gehc_ytd.groupby('Year of service_close_date')[
                'Total GE Cost'].sum()
            if len(gehc_ytd_group) > 0:
                score_card.loc[index, str(
                    gehc_ytd_group.index.get_level_values(0)[0]) + ' YTD'] = \
                    gehc_ytd_group[gehc_ytd_group.index.get_level_values(0)[0]]

        elif row['Product Quality Scorecard'] != 'GEHC' and row[
            'Business'] == 'ALL' and row['Modality'] == 'ALL':
            business = data.loc[data['SC Business Segment'] == row[
                'Product Quality Scorecard']]
            business_group = business.groupby('Year of service_close_date')[
                'Total GE Cost'].sum()
            for value in business_group.index.get_level_values(0):
                if value != current_year:
                    score_card.loc[index, str(value) + ' Total'] = \
                        business_group[value]
                else:
                    score_card.loc[index, str(value) + ' YTD'] = \
                        business_group[value]
            business_ytd = business[
                ['Total GE Cost', 'Year of service_close_date',
                 'Week Number']].loc[
                (
                        business[
                            'Year of service_close_date'] == current_year - 1) & (
                        business['Week Number'] <= weekNumber)]
            business_ytd_group = \
                business_ytd.groupby('Year of service_close_date')[
                    'Total GE Cost'].sum()
            if len(business_ytd_group) > 0:
                score_card.loc[index, str(
                    business_ytd_group.index.get_level_values(0)[0]) + ' YTD'] = \
                    business_ytd_group[
                        business_ytd_group.index.get_level_values(0)[0]]
        elif row['Business'] != 'ALL' and row['Modality'] == 'ALL' and row[
            'Product Quality Scorecard'] != 'LCS':
            modality = \
                data[['SC Business Segment', 'SC Modality', 'Total GE Cost',
                      'Year of service_close_date',
                      'Week Number']].loc[
                    (data['SC Business Segment'] == row['Business']) & (
                            data['SC Modality'] == row[
                        'Product Quality Scorecard'])]
            modality_group = modality.groupby('Year of service_close_date')[
                'Total GE Cost'].sum()
            for value in modality_group.index.get_level_values(0):
                if value != current_year:
                    score_card.loc[index, str(value) + ' Total'] = \
                        modality_group[value]
                else:
                    score_card.loc[index, str(value) + ' YTD'] = \
                        modality_group[value]
            modality_ytd = modality[
                ['Total GE Cost', 'Year of service_close_date',
                 'Week Number']].loc[
                (
                        modality[
                            'Year of service_close_date'] == current_year - 1) & (
                        modality['Week Number'] <= weekNumber)]
            modality_ytd_group = \
                modality_ytd.groupby('Year of service_close_date')[
                    'Total GE Cost'].sum()
            if len(modality_ytd_group) > 0:
                score_card.loc[index, str(
                    modality_ytd_group.index.get_level_values(0)[0]) + ' YTD'] = \
                    modality_ytd_group[
                        modality_ytd_group.index.get_level_values(0)[0]]
        elif row['Business'] != 'ALL' and row['Modality'] == 'ALL' and row[
            'Product Quality Scorecard'] == 'LCS':
            modality = \
                data[['SC Business Segment', 'SC Modality', 'Total GE Cost',
                      'Year of service_close_date',
                      'Week Number']].loc[
                    (data['SC Business Segment'] == row['Business'])
                    & (data['SC Modality'] != 'ULTRASOUND')
                    & (data['SC Modality'] != 'EXCLUDED')
                    & (data['SC Modality'] != 'OTHER')]
            modality_group = modality.groupby('Year of service_close_date')[
                'Total GE Cost'].sum()
            for value in modality_group.index.get_level_values(0):
                if value != current_year:
                    score_card.loc[index, str(value) + ' Total'] = \
                        modality_group[value]
                else:
                    score_card.loc[index, str(value) + ' YTD'] = \
                        modality_group[value]
            modality_ytd = modality[
                ['Total GE Cost', 'Year of service_close_date',
                 'Week Number']].loc[
                (
                        modality[
                            'Year of service_close_date'] == current_year - 1) & (
                        modality['Week Number'] <= weekNumber)]
            modality_ytd_group = \
                modality_ytd.groupby('Year of service_close_date')[
                    'Total GE Cost'].sum()
            if len(modality_ytd_group) > 0:
                score_card.loc[index, str(
                    modality_ytd_group.index.get_level_values(0)[0]) + ' YTD'] = \
                    modality_ytd_group[
                        modality_ytd_group.index.get_level_values(0)[0]]
        elif row['Business'] != 'ALL' and row['Modality'] != 'ALL' and row[
            'Product Segment'] == 'ALL':
            prod_seg = \
                data[['SC Business Segment', 'SC Modality',
                      'SC Product Segment', 'Total GE Cost',
                      'Year of service_close_date', 'Week Number']].loc[
                    (data['SC Business Segment'] == row['Business']) & (
                            data['SC Modality'] == row['Modality']) & (
                            data['SC Product Segment'] == row[
                        'Product Quality Scorecard'])]
            prod_seg_group = prod_seg.groupby('Year of service_close_date')[
                'Total GE Cost'].sum()
            for value in prod_seg_group.index.get_level_values(0):
                if value != current_year:
                    score_card.loc[index, str(value) + ' Total'] = \
                        prod_seg_group[value]
                else:
                    score_card.loc[index, str(value) + ' YTD'] = \
                        prod_seg_group[value]
            try:
                prod_seg_ytd = prod_seg[
                    ['Total GE Cost', 'Year of service_close_date',
                     'Week Number']].loc[
                    (prod_seg[
                         'Year of service_close_date'] == current_year - 1) & (
                            prod_seg['Week Number'] <= weekNumber)]

                prod_seg_ytd_group = \
                    prod_seg_ytd.groupby('Year of service_close_date')[
                        'Total GE Cost'].sum()

                score_card.loc[index, str(
                    prod_seg_ytd_group.index.get_level_values(0)[
                        0]) + ' YTD'] = \
                    prod_seg_ytd_group[
                        prod_seg_ytd_group.index.get_level_values(0)[0]]
            except KeyError:
                score_card.loc[index, str(current_year - 1) + ' YTD'] = ''
            except IndexError:
                score_card.loc[index, str(current_year - 1) + ' YTD'] = ''
        

    # score_card.drop(columns=['Business', 'Modality'], inplace=True)
    return score_card

In [3]:
def calculate_cpu_pqm(score_card, data):
    data['Measure Names'] = '# of Complaints'
    current_year = datetime.datetime.now().year
    current_month = datetime.datetime.now().month
    for index, row in score_card.iterrows():
        if row['Product Quality Scorecard'] == 'GEHC':
            gehc_complaint = data[['Year', 'Measure Names', 'Measure Values']].loc[data['Measure Names'] == '# of Complaints'].groupby('Year')['Measure Values'].sum()
                    

            complaint_rate = gehc_complaint

            for value in complaint_rate.index.get_level_values(0):
                if int(value) != current_year:
                    score_card.loc[index, str(value) + ' Total'] = complaint_rate[value] 
                else:
                    #print(gehc_ib_sum_cy["Measure Values"].loc[gehc_ib_sum_cy['Year'] == value])
                    score_card.loc[index, str((value))  + ' YTD'] = complaint_rate[value] 

            gehc_complaint_ytd = data[['Year', 'Measure Names', 'Measure Values']].loc[\
                (data['Year'] == current_year - 1) & (data['Month'] <= current_month - 1)\
                & (data['Measure Names'] == '# of Complaints')].groupby(['Year'])['Measure Values']\
                                                .sum()

            complaint_rate_ytd = gehc_complaint_ytd 

            try:
                score_card.loc[index, str(int(complaint_rate_ytd.index.get_level_values(0)[0])) + ' YTD'] = \
                    complaint_rate_ytd[complaint_rate_ytd.index.get_level_values(0)[0]]
                
            except IndexError:
                score_card.loc[index, str(current_year - 1) + ' YTD'] = ''
                

        elif row['Product Quality Scorecard'] != 'GEHC' and row['Business'] == 'All' and row['Modality'] == 'All':
            business = data.loc[data['SC Business Segment'] == row['Product Quality Scorecard']]
            complaint_business = business[['Year', 'Measure Names', 'Measure Values']].loc[\
                business['Measure Names'] == '# of Complaints'].groupby('Year')['Measure Values'].sum()
            
            complaint_rate_business = complaint_business 

            for value in complaint_rate_business.index.get_level_values(0):
                print(complaint_business[value])
                if int(value) != current_year:
                    score_card.loc[index, str(value) + ' Total'] = complaint_rate_business[value]  
                else:
                    score_card.loc[index, str(int(value))  + ' YTD'] = complaint_rate_business[value] 

            business_complaint_ytd = business[['Year', 'Month', 'Measure Names', 'Measure Values']].loc[\
                (business['Year'] == current_year - 1) & (business['Month'] <= current_month - 1)\
                & (business['Measure Names'] == '# of Complaints')].groupby('Year')['Measure Values'].sum()

            
            business_complaint_rate_ytd = business_complaint_ytd 

            try:
                score_card.loc[index, str(int(business_complaint_rate_ytd.index.get_level_values(0)[0])) + ' YTD'] = \
                    business_complaint_rate_ytd[business_complaint_rate_ytd.index.get_level_values(0)[0]]
                

            except IndexError:
                score_card.loc[index, str(current_year - 1) + ' YTD'] = ''
                

        elif row['Business'] != 'All' and row['Modality'] == 'All':
            modality = data[['SC Business Segment', 'SC Modality', 'Month', 'Year', 'Measure Names',
                             'Measure Values']].loc[(data['SC Business Segment'] == row['Business']) & (
                    data['SC Modality'] == row['Product Quality Scorecard'])]

            complaint_modality = modality[['Year', 'Measure Names', 'Measure Values']].loc[\
                modality['Measure Names'] == '# of Complaints'].groupby('Year')['Measure Values'].sum()

    

            complaint_rate_modality = complaint_modality 

            for value in complaint_rate_modality.index.get_level_values(0):
                if int(value) != current_year:
                    score_card.loc[index, str(value) + ' Total'] = complaint_rate_modality[value] 
                else:
                    score_card.loc[index, str(int(value))  + ' YTD'] = complaint_rate_modality[value] 
                    
            modality_complaint_ytd = modality[['Year', 'Month', 'Measure Names', 'Measure Values']].loc[\
                (modality['Year'] == current_year - 1) & (modality['Month'] <= current_month - 1)\
                & (modality['Measure Names'] == '# of Complaints')].groupby('Year')['Measure Values'].sum()

            
            modality_complaint_rate_ytd = modality_complaint_ytd 
            try:
                score_card.loc[index, str(int(modality_complaint_rate_ytd.index.get_level_values(0)[0])) + ' YTD'] = \
                    modality_complaint_rate_ytd[modality_complaint_rate_ytd.index.get_level_values(0)[0]]
                
            except IndexError:
                score_card.loc[index, str(current_year - 1) + ' YTD'] = ''
        

        elif row['Business'] != 'All' and row['Modality'] != 'All' and row['Product Segment'] == 'All':
            prod_seg = \
                data[['SC Business Segment', 'SC Modality', 'SC Product Segment',
                      'Year', 'Measure Names', 'Measure Values', 'Month']].loc[(data['SC Business Segment'] == row['Business']) & (data['SC Modality'] ==
                                                                        row['Modality']) & (
                        data['SC Product Segment'] == row['Product Quality Scorecard'])]

            complaint_prod_seg = prod_seg[['Year', 'Measure Names', 'Measure Values']].loc[\
                prod_seg['Measure Names'] == '# of Complaints'].groupby('Year')['Measure Values'].sum()


            complaint_rate_prod_seg = complaint_prod_seg 

            for value in complaint_prod_seg.index.get_level_values(0):
                if int(value) != current_year:
                    score_card.loc[index, str(value) + ' Total'] = complaint_rate_prod_seg[value] 
                else:
                    score_card.loc[index, str(int(value))  + ' YTD'] = complaint_rate_prod_seg[value] 


            prod_seg_complaint_ytd = prod_seg[['Year', 'Month', 'Measure Names', 'Measure Values']].loc[\
                (prod_seg['Year'] == current_year - 1) & (prod_seg['Month'] <= current_month - 1)\
                & (prod_seg['Measure Names'] == '# of Complaints')].groupby('Year')['Measure Values'].sum()

            
            prod_seg_complaint_rate_ytd = prod_seg_complaint_ytd 
            try:
                score_card.loc[index, str(int(prod_seg_complaint_rate_ytd.index.get_level_values(0)[0])) + ' YTD'] = \
                    prod_seg_complaint_rate_ytd[prod_seg_complaint_rate_ytd.index.get_level_values(0)[0]]
                
            except IndexError:
                score_card.loc[index, str(current_year - 1) + ' YTD'] = ''
           #         else:
#             product = \
#                 data[['SC Business Segment', 'SC Modality', 'SC Product Segment', 'SC Product',
#                       'Year', 'Measure Names', 'Measure Values', 'Month']].loc[(data['SC Business Segment'] == row['Business']) & (data['SC Modality'] ==
#                                                                         row['Modality']) & (
#                         data['SC Product Segment'] == row['Product Segment']) &(data['SC Product'] == row['Product Quality Scorecard'])]

#             complaint_prod = product[['Year', 'Measure Names', 'Measure Values']].loc[\
#                 product['Measure Names'] == '# of Complaints'].groupby(['Year'])['Measure Values'].sum()
            
            
#             complaint_rate_prod = complaint_prod 


#             for value in complaint_rate_prod.index.get_level_values(0):
#                 if int(value) != current_year:
#                     score_card.loc[index, str(int(value))  + ' Total'] = complaint_rate_prod[value]
#                 else:
#                     score_card.loc[index, str(int(value))  + ' YTD'] = complaint_rate_prod[value] 

                    
#             prod_complaint_ytd = product[['Year', 'Month', 'Measure Names', 'Measure Values']].loc[\
#                 (product['Year'] == current_year - 1) & (product['Month'] <= current_month - 1)\
#                 & (product['Measure Names'] == '# of Complaints')].groupby('Year')['Measure Values'].sum()
            
            
#             prod_complaint_rate_ytd = prod_complaint_ytd 


#             try:
#                 score_card.loc[index, str(int(prod_complaint_rate_ytd.index.get_level_values(0)[0])) + ' YTD'] = \
#                     prod_complaint_rate_ytd[prod_complaint_rate_ytd.index.get_level_values(0)[0]]
                
#             except IndexError:
#                 score_card.loc[index, str(current_year - 1) + ' YTD'] = ''
           

    return score_card

In [4]:
def fill_in_sc_data(combined_sc, scorecard_value_format, business_value_format, modality_value_format, otd,
                    oti, cso, worksheet, ifr, ib_data, comp_pqm, obf):
# def fill_in_sc_data(combined_sc, scorecard_value_format, business_value_format, modality_value_format, complaints, otd,
#                     oti, cso,cso_o, worksheet, ifr):
    book = xlrd.open_workbook('/Users/503056565/Data/Customer/scorecard calculations/Customer_Scorecard.xlsx')
    sheet = book.sheet_by_index(0)
    # print(combined_sc)
#     for index, row in combined_sc.iterrows():
#         style = ''
#         if row['Product Quality Scorecard'] == 'GEHC':
#             style = scorecard_value_format
#         elif row['Product Quality Scorecard'] != 'GEHC' and row['Business'] == 'All' and row['Modality'] == 'All' and \
#                         row['Product Segment'] == 'All':
#             style = business_value_format
#         elif row['Product Quality Scorecard'] != 'GEHC' and row['Business'] != 'All' and row['Modality'] == 'All' and \
#                         row['Product Segment'] == 'All':
#             style = modality_value_format
#         elif row['Product Quality Scorecard'] != 'GEHC' and row['Business'] != 'All' and row['Modality'] != 'All' and \
#                         row['Product Segment'] == 'All':
#             style = scorecard_value_format
#         else:
#             style = scorecard_value_format
#         complaint_data = complaints.loc[(complaints['Product Quality Scorecard'] == row['Product Quality Scorecard']) & \
#                                         (complaints['Business'] == row['Business']) \
#                                         & (complaints['Modality'] == row['Modality']) & \
#                                         (complaints['Product Segment'] == row['Product Segment'])]

#         rownum = 0
#         for row_index in range(sheet.nrows):
#             if sheet.cell_value(row_index, 0) == row['Product Quality Scorecard'] and \
#                             sheet.cell_value(row_index, 1) == row['Business'] and \
#                             sheet.cell_value(row_index, 2) == row['Modality'] and \
#                             sheet.cell_value(row_index, 3) == row['Product Segment']:
#                 #print(row['Product Quality Scorecard'])
#                 rownum = row_index

#         if complaint_data.size > 0:
#             worksheet.write('E' + str(rownum + 1), complaint_data['2017 Total'].values[0], style)
#             worksheet.write('F' + str(rownum + 1), complaint_data['2018 YTD'].values[0], style)
#             worksheet.write('G' + str(rownum + 1), complaint_data['2018 YTD IB Installs'].values[0], style)
#             worksheet.write('H' + str(rownum + 1), complaint_data['2019 YTD'].values[0], style)
#             worksheet.write('I' + str(rownum + 1), complaint_data['2019 YTD IB Installs'].values[0], style)
#             worksheet.write('J' + str(rownum + 1), complaint_data['2018 YTD IB Complaints'].values[0], style)
#             worksheet.write('K' + str(rownum + 1), complaint_data['2019 YTD IB Complaints'].values[0], style)
#         else:
#             worksheet.write('E' + str(rownum + 1), 'n/a', style)
#             worksheet.write('F' + str(rownum + 1), 'n/a', style)
#             worksheet.write('G' + str(rownum + 1), 'n/a', style)
#             worksheet.write('H' + str(rownum + 1), 'n/a', style)
#             worksheet.write('I' + str(rownum + 1), 'n/a', style)
#             worksheet.write('J' + str(rownum + 1), 'n/a', style)
#             worksheet.write('K' + str(rownum + 1), 'n/a', style)
  
    otd.fillna('', inplace=True)

    for index, row in combined_sc.iterrows():
        style = ''
        if row['Product Quality Scorecard'] == 'GEHC':
            style = scorecard_value_format
        elif row['Product Quality Scorecard'] != 'GEHC' and row['Business'] == 'All' and row['Modality'] == 'All' and \
                        row['Product Segment'] == 'All':
            style = business_value_format
        elif row['Product Quality Scorecard'] != 'GEHC' and row['Business'] != 'All' and row['Modality'] == 'All' and \
                        row['Product Segment'] == 'All':
            style = modality_value_format
        elif row['Product Quality Scorecard'] != 'GEHC' and row['Business'] != 'All' and row['Modality'] != 'All' and \
                        row['Product Segment'] == 'All':
            style = scorecard_value_format
        else:
            style = scorecard_value_format
        otd_data = otd.loc[(otd['Product Quality Scorecard'] == row['Product Quality Scorecard']) & \
                           (otd['Business'] == row['Business']) \
                           & (otd['Modality'] == row['Modality']) & \
                           (otd['Product Segment'] == row['Product Segment'])]

        rownum = -1
        for row_index in range(sheet.nrows):
            if sheet.cell_value(row_index, 0) == row['Product Quality Scorecard'] and \
                            sheet.cell_value(row_index, 1) == row['Business'] and \
                            sheet.cell_value(row_index, 2) == row['Modality'] and \
                            sheet.cell_value(row_index, 3) == row['Product Segment']:
                print(row['Product Quality Scorecard'])
                rownum = row_index

        if otd_data.size > 0:
            if otd_data['2018 Total'].values[0] == '':
                worksheet.write('E' + str(rownum + 1), otd_data['2018 Total'].values[0], style)
            else:
                worksheet.write('E' + str(rownum + 1), str(round(float(otd_data['2018 Total'].values[0]))) + '%', style)

            if otd_data['2018 YTD'].values[0] == '':
                worksheet.write('F' + str(rownum + 1), otd_data['2018 YTD'].values[0], style)
            else:
                worksheet.write('F' + str(rownum + 1), str(round(float(otd_data['2018 YTD'].values[0]))) + '%', style)
            
            if otd_data['2018 YTD Installs'].values[0] == '':
                worksheet.write('G' + str(rownum + 1), otd_data['2018 YTD Installs'].values[0], style)
            else:
                worksheet.write('G' + str(rownum + 1), str(round(float(otd_data['2018 YTD Installs'].values[0]))) , style)

            if otd_data['2019 YTD'].values[0] == '':
                worksheet.write('H' + str(rownum + 1), otd_data['2019 YTD'].values[0], style)
            else:
                worksheet.write('H' + str(rownum + 1), str(round(float(otd_data['2019 YTD'].values[0]))) + '%', style)
            
            if otd_data['2019 YTD Installs'].values[0] == '':
                worksheet.write('I' + str(rownum + 1), otd_data['2019 YTD Installs'].values[0], style)
            else:
                worksheet.write('I' + str(rownum + 1), str(round(float(otd_data['2019 YTD Installs'].values[0]))) , style)
        else:
            worksheet.write('E' + str(rownum + 1), '', style)
            worksheet.write('F' + str(rownum + 1), '', style)
            worksheet.write('G' + str(rownum + 1), '', style)
            worksheet.write('H' + str(rownum + 1), '', style)
            worksheet.write('I' + str(rownum + 1), '', style)


    # On Time Install
    oti.fillna('', inplace=True)
    
    #oti_totalinstalls.fillna('n/a', inplace=True)

    for index, row in combined_sc.iterrows():
        style = ''
        if row['Product Quality Scorecard'] == 'GEHC':
            style = scorecard_value_format
        elif row['Product Quality Scorecard'] != 'GEHC' and row['Business'] == 'All' and row['Modality'] == 'All' and \
                        row['Product Segment'] == 'All':
            style = business_value_format
        elif row['Product Quality Scorecard'] != 'GEHC' and row['Business'] != 'All' and row['Modality'] == 'All' and \
                        row['Product Segment'] == 'All':
            style = modality_value_format
        elif row['Product Quality Scorecard'] != 'GEHC' and row['Business'] != 'All' and row['Modality'] != 'All' and \
                        row['Product Segment'] == 'All':
            style = scorecard_value_format
        else:
            style = scorecard_value_format
        oti_data = oti.loc[(oti['Product Quality Scorecard'] == row['Product Quality Scorecard']) & \
                           (oti['Business'] == row['Business']) \
                           & (oti['Modality'] == row['Modality']) & \
                           (oti['Product Segment'] == row['Product Segment'])]

        rownum = -1
        for row_index in range(sheet.nrows):
            if sheet.cell_value(row_index, 0) == row['Product Quality Scorecard'] and \
                            sheet.cell_value(row_index, 1) == row['Business'] and \
                            sheet.cell_value(row_index, 2) == row['Modality'] and \
                            sheet.cell_value(row_index, 3) == row['Product Segment']:
                #print(row['Product Quality Scorecard'])
                rownum = row_index

        if (oti_data.size > 0):#stalls.size>0):

            if oti_data['2018 Total'].values[0] == '':
                worksheet.write('J' + str(rownum + 1), oti_data['2018 Total'].values[0], style)
            else:
                worksheet.write('J' + str(rownum + 1), str(round(float(oti_data['2018 Total'].values[0]))) + '%', style)

            if oti_data['2018 YTD'].values[0] == '':
                worksheet.write('K' + str(rownum + 1), oti_data['2018 YTD'].values[0], style)
            else:
                worksheet.write('K' + str(rownum + 1), str(round(float(oti_data['2018 YTD'].values[0]))) + '%', style)
            
            if oti_data['2018 YTD Installs'].values[0] == '':
                worksheet.write('L' + str(rownum + 1), oti_data['2018 YTD Installs'].values[0], style)
            else:
                worksheet.write('L' + str(rownum + 1), str(round(float(oti_data['2018 YTD Installs'].values[0]))), style)

            if oti_data['2019 YTD'].values[0] == '':
                worksheet.write('M' + str(rownum + 1), oti_data['2019 YTD'].values[0], style)
            else:
                worksheet.write('M' + str(rownum + 1), str(round(float(oti_data['2019 YTD'].values[0]))) + '%', style)
            
            if oti_data['2019 YTD Installs'].values[0] == '':
                worksheet.write('N' + str(rownum + 1), oti_data['2019 YTD Installs'].values[0], style)
            else:
                worksheet.write('N' + str(rownum + 1), str(round(float(oti_data['2019 YTD Installs'].values[0]))), style)
                
        else:
            worksheet.write('J' + str(rownum + 1), '', style)
            worksheet.write('K' + str(rownum + 1), '', style)
            worksheet.write('L' + str(rownum + 1), '', style)
            worksheet.write('M' + str(rownum + 1), '', style)
            worksheet.write('N' + str(rownum + 1), '', style)
    
    

    

    # IFR90
    ifr.fillna('', inplace=True)

    for index, row in combined_sc.iterrows():
        style = ''
        if row['Product Quality Scorecard'] == 'GEHC':
            style = scorecard_value_format
        elif row['Product Quality Scorecard'] != 'GEHC' and row['Business'] == 'All' and row[
            'Modality'] == 'All' and \
                        row['Product Segment'] == 'All':
            style = business_value_format
        elif row['Product Quality Scorecard'] != 'GEHC' and row['Business'] != 'All' and row[
            'Modality'] == 'All' and \
                        row['Product Segment'] == 'All':
            style = modality_value_format
        elif row['Product Quality Scorecard'] != 'GEHC' and row['Business'] != 'All' and row[
            'Modality'] != 'All' and \
                        row['Product Segment'] == 'All':
            style = scorecard_value_format
        else:
            style = scorecard_value_format
        ifr_data = ifr.loc[(ifr['Product Quality Scorecard'] == row['Product Quality Scorecard']) & \
                           (ifr['Business'] == row['Business']) \
                           & (ifr['Modality'] == row['Modality']) & \
                           (ifr['Product Segment'] == row['Product Segment'])]

        rownum = -1
        for row_index in range(sheet.nrows):
            if sheet.cell_value(row_index, 0) == row['Product Quality Scorecard'] and \
                            sheet.cell_value(row_index, 1) == row['Business'] and \
                            sheet.cell_value(row_index, 2) == row['Modality'] and \
                            sheet.cell_value(row_index, 3) == row['Product Segment']:
                #print(row['Product Quality Scorecard'])
                rownum = row_index

        if ifr_data.size > 0:

            if ifr_data['2018 Total'].values[0] == '':
                worksheet.write('O' + str(rownum + 1), ifr_data['2018 Total'].values[0], style)
            else:
                worksheet.write('O' + str(rownum + 1), str(round(float(ifr_data['2018 Total'].values[0]),2)),
                                style)

            if ifr_data['2018 YTD'].values[0] == '':
                worksheet.write('P' + str(rownum + 1), ifr_data['2018 YTD'].values[0], style)
            else:
                worksheet.write('P' + str(rownum + 1), str(round(float(ifr_data['2018 YTD'].values[0]),2)),
                                style)
                
            if ifr_data['2018 YTD Installs'].values[0] == '':
                worksheet.write('Q' + str(rownum + 1), ifr_data['2018 YTD Installs'].values[0], style)
            else:
                worksheet.write('Q' + str(rownum + 1), str(round(float(ifr_data['2018 YTD Installs'].values[0]),2)),
                                style)

            if ifr_data['2019 YTD'].values[0] == '':
                worksheet.write('R' + str(rownum + 1), ifr_data['2019 YTD'].values[0], style)
            else:
                worksheet.write('R' + str(rownum + 1), str(round(float(ifr_data['2019 YTD'].values[0]),2)),
                                style)
            if ifr_data['2019 YTD Installs'].values[0] == '':
                worksheet.write('S' + str(rownum + 1), ifr_data['2019 YTD Installs'].values[0], style)
            else:
                worksheet.write('S' + str(rownum + 1), str(round(float(ifr_data['2019 YTD Installs'].values[0]),2)),
                                style)
        else:
            worksheet.write('O' + str(rownum + 1), '', style)
            worksheet.write('P' + str(rownum + 1), '', style)
            worksheet.write('Q' + str(rownum + 1), '', style)
            worksheet.write('R' + str(rownum + 1), '', style)
            worksheet.write('S' + str(rownum + 1), '', style)
    # CSO
    cso.fillna('', inplace=True)

    for index, row in combined_sc.iterrows():
        style = ''
        if row['Product Quality Scorecard'] == 'GEHC':
            style = scorecard_value_format
        elif row['Product Quality Scorecard'] != 'GEHC' and row['Business'] == 'All' and row['Modality'] == 'All' and \
                        row['Product Segment'] == 'All':
            style = business_value_format
        elif row['Product Quality Scorecard'] != 'GEHC' and row['Business'] != 'All' and row['Modality'] == 'All' and \
                        row['Product Segment'] == 'All':
            style = modality_value_format
        elif row['Product Quality Scorecard'] != 'GEHC' and row['Business'] != 'All' and row['Modality'] != 'All' and \
                        row['Product Segment'] == 'All':
            style = scorecard_value_format
        else:
            style = scorecard_value_format
        cso_data = cso.loc[(cso['Product Quality Scorecard'] == row['Product Quality Scorecard']) & \
                           (cso['Business'] == row['Business']) \
                           & (cso['Modality'] == row['Modality']) & \
                           (cso['Product Segment'] == row['Product Segment'])]

        rownum = -1
        for row_index in range(sheet.nrows):
            if sheet.cell_value(row_index, 0) == row['Product Quality Scorecard'] and \
                            sheet.cell_value(row_index, 1) == row['Business'] and \
                            sheet.cell_value(row_index, 2) == row['Modality'] and \
                            sheet.cell_value(row_index, 3) == row['Product Segment']:
                #print(row['Product Quality Scorecard'])
                rownum = row_index

        if cso_data.size > 0:

            worksheet.write('T' + str(rownum + 1), cso_data['2018 Total'].values[0], style)
            worksheet.write('U' + str(rownum + 1), cso_data['2019 YTD'].values[0], style)

        else:
            worksheet.write('T' + str(rownum + 1), '', style)
            worksheet.write('U' + str(rownum + 1), '', style)
            
#     # CSO Open
#     cso_o.fillna('n/a', inplace=True)

#     for index, row in combined_sc.iterrows():
#         style = ''
#         if row['Product Quality Scorecard'] == 'GEHC':
#             style = scorecard_value_format
#         elif row['Product Quality Scorecard'] != 'GEHC' and row['Business'] == 'All' and row['Modality'] == 'All' and \
#                         row['Product Segment'] == 'All':
#             style = business_value_format
#         elif row['Product Quality Scorecard'] != 'GEHC' and row['Business'] != 'All' and row['Modality'] == 'All' and \
#                         row['Product Segment'] == 'All':
#             style = modality_value_format
#         elif row['Product Quality Scorecard'] != 'GEHC' and row['Business'] != 'All' and row['Modality'] != 'All' and \
#                         row['Product Segment'] == 'All':
#             style = scorecard_value_format
#         else:
#             style = scorecard_value_format
#         cso_data_o = cso_o.loc[(cso_o['Product Quality Scorecard'] == row['Product Quality Scorecard']) & \
#                            (cso_o['Business'] == row['Business']) \
#                            & (cso_o['Modality'] == row['Modality']) & \
#                            (cso_o['Product Segment'] == row['Product Segment'])]

#         rownum = 0
#         for row_index in range(sheet.nrows):
#             if sheet.cell_value(row_index, 0) == row['Product Quality Scorecard'] and \
#                             sheet.cell_value(row_index, 1) == row['Business'] and \
#                             sheet.cell_value(row_index, 2) == row['Modality'] and \
#                             sheet.cell_value(row_index, 3) == row['Product Segment']:
#                 #print(row['Product Quality Scorecard'])
#                 rownum = row_index

#         if cso_data_o.size > 0:

#             worksheet.write('AC' + str(rownum + 1), cso_data_o['2017 Total'].values[0], style)
#             worksheet.write('AD' + str(rownum + 1), cso_data_o['2018 YTD'].values[0], style)
#             worksheet.write('AE' + str(rownum + 1), cso_data_o['2019 YTD'].values[0], style)

#         else:
#             worksheet.write('AC' + str(rownum + 1), 'n/a', style)
#             worksheet.write('AD' + str(rownum + 1), 'n/a', style)
#             worksheet.write('AE' + str(rownum + 1), 'n/a', style)
      
    ib_data.fillna('', inplace=True)

    for index, row in combined_sc.iterrows():
        style = ''
        if row['Product Quality Scorecard'] == 'GEHC':
            style = scorecard_value_format
        elif row['Product Quality Scorecard'] != 'GEHC' and row['Business'] == 'All' and row['Modality'] == 'All' and \
                        row['Product Segment'] == 'All':
            style = business_value_format
        elif row['Product Quality Scorecard'] != 'GEHC' and row['Business'] != 'All' and row['Modality'] == 'All' and \
                        row['Product Segment'] == 'All':
            style = modality_value_format
        elif row['Product Quality Scorecard'] != 'GEHC' and row['Business'] != 'All' and row['Modality'] != 'All' and \
                        row['Product Segment'] == 'All':
            style = scorecard_value_format
        else:
            style = scorecard_value_format
        ib_data_o = ib_data.loc[(ib_data['Product Quality Scorecard'] == row['Product Quality Scorecard']) & \
                           (ib_data['Business'] == row['Business']) \
                           & (ib_data['Modality'] == row['Modality']) & \
                           (ib_data['Product Segment'] == row['Product Segment'])]

        rownum = -1
        for row_index in range(sheet.nrows):
            if sheet.cell_value(row_index, 0) == row['Product Quality Scorecard'] and \
                            sheet.cell_value(row_index, 1) == row['Business'] and \
                            sheet.cell_value(row_index, 2) == row['Modality'] and \
                            sheet.cell_value(row_index, 3) == row['Product Segment']:
                #print(row['Product Quality Scorecard'])
                rownum = row_index

        if ib_data_o.size > 0:

            worksheet.write('V' + str(rownum + 1), ib_data_o['2018 Total'].values[0], style)
            worksheet.write('W' + str(rownum + 1), ib_data_o['2018 YTD'].values[0], style)
            worksheet.write('X' + str(rownum + 1), ib_data_o['2019 YTD'].values[0], style)

        else:
            worksheet.write('V' + str(rownum + 1), '', style)
            worksheet.write('W' + str(rownum + 1), '', style)
            worksheet.write('X' + str(rownum + 1), '', style)
    
    #Comp_pqm
    
    comp_pqm.fillna('', inplace=True)

    for index, row in combined_sc.iterrows():
        style = ''
        if row['Product Quality Scorecard'] == 'GEHC':
            style = scorecard_value_format
        elif row['Product Quality Scorecard'] != 'GEHC' and row['Business'] == 'All' and row['Modality'] == 'All' and \
                        row['Product Segment'] == 'All':
            style = business_value_format
        elif row['Product Quality Scorecard'] != 'GEHC' and row['Business'] != 'All' and row['Modality'] == 'All' and \
                        row['Product Segment'] == 'All':
            style = modality_value_format
        elif row['Product Quality Scorecard'] != 'GEHC' and row['Business'] != 'All' and row['Modality'] != 'All' and \
                        row['Product Segment'] == 'All':
            style = scorecard_value_format
        else:
            style = scorecard_value_format
        comp_pqm_o = comp_pqm.loc[(comp_pqm['Product Quality Scorecard'] == row['Product Quality Scorecard']) & \
                           (comp_pqm['Business'] == row['Business']) \
                           & (comp_pqm['Modality'] == row['Modality']) & \
                           (comp_pqm['Product Segment'] == row['Product Segment'])]

        rownum = -1
        for row_index in range(sheet.nrows):
            if sheet.cell_value(row_index, 0) == row['Product Quality Scorecard'] and \
                            sheet.cell_value(row_index, 1) == row['Business'] and \
                            sheet.cell_value(row_index, 2) == row['Modality'] and \
                            sheet.cell_value(row_index, 3) == row['Product Segment']:
                #print(row['Product Quality Scorecard'])
                rownum = row_index

        if comp_pqm_o.size > 0:

            worksheet.write('Y' + str(rownum + 1), comp_pqm_o['2018 Total'].values[0], style)
            worksheet.write('Z' + str(rownum + 1), comp_pqm_o['2018 YTD'].values[0], style)
            worksheet.write('AA' + str(rownum + 1), comp_pqm_o['2019 YTD'].values[0], style)

        else:
            worksheet.write('Y' + str(rownum + 1), '', style)
            worksheet.write('Z' + str(rownum + 1), '', style)
            worksheet.write('AA' + str(rownum + 1), '', style)
            
     #obf
    
    obf.fillna('', inplace=True)

    for index, row in combined_sc.iterrows():
        style = ''
        if row['Product Quality Scorecard'] == 'GEHC':
            style = scorecard_value_format
        elif row['Product Quality Scorecard'] != 'GEHC' and row['Business'] == 'All' and row['Modality'] == 'All' and \
                        row['Product Segment'] == 'All':
            style = business_value_format
        elif row['Product Quality Scorecard'] != 'GEHC' and row['Business'] != 'All' and row['Modality'] == 'All' and \
                        row['Product Segment'] == 'All':
            style = modality_value_format
        elif row['Product Quality Scorecard'] != 'GEHC' and row['Business'] != 'All' and row['Modality'] != 'All' and \
                        row['Product Segment'] == 'All':
            style = scorecard_value_format
        else:
            style = scorecard_value_format
        obf_o = obf.loc[(obf['Product Quality Scorecard'] == row['Product Quality Scorecard']) & \
                           (obf['Business'] == row['Business']) \
                           & (obf['Modality'] == row['Modality']) & \
                           (obf['Product Segment'] == row['Product Segment'])]

        rownum = -1
        for row_index in range(sheet.nrows):
            if sheet.cell_value(row_index, 0) == row['Product Quality Scorecard'] and \
                            sheet.cell_value(row_index, 1) == row['Business'] and \
                            sheet.cell_value(row_index, 2) == row['Modality'] and \
                            sheet.cell_value(row_index, 3) == row['Product Segment']:
                #print(row['Product Quality Scorecard'])
                rownum = row_index

        if obf_o.size > 0:

            worksheet.write('AB' + str(rownum + 1), obf_o['2018 Total'].values[0], style)
            worksheet.write('AC' + str(rownum + 1), obf_o['2018 YTD'].values[0], style)
            worksheet.write('AD' + str(rownum + 1), obf_o['2019 YTD'].values[0], style)

        else:
            worksheet.write('AB' + str(rownum + 1), '', style)
            worksheet.write('AC' + str(rownum + 1), '', style)
            worksheet.write('AD' + str(rownum + 1), '', style)            
    

# Creating customer scorecard

In [27]:
sii_hierarchy = pd.read_excel('/Users/sowmiyanmorri/Documents/Balance Scorecard/Deepa Scorecard Files/Customer/configuration/SII Hierarchy.xlsx')
print("done")
# ##OTD (ok) use Mayo file
# otd = pd.read_excel('/Users/503056565/Data/Customer/source files/OTD.xlsx')
# otd_modality = pd.read_excel('/Users/503056565/Data/Customer/configuration/OTD Modalities.xlsx')
# otd = setup_sii_sc_hierarchy(otd,sii_hierarchy,otd_modality, 'OTD')

# otd.to_csv('/Users/503056565/Data/Customer/scorecard calculations/OTD_Calculations.csv',index=False)
# otd_score_card = setup_scorecard_structure(otd)
# otd_score_card_final = calculate_otd(otd_score_card,otd)
# otd_score_card_final.to_csv('/Users/503056565/Data/Customer/scorecard calculations/OTD_SC.csv',index=False)

# ##OTI (ok) use Mayo file
# oti_modality = pd.read_excel('/Users/503056565/Data/Customer/configuration/OTI Modalities.xlsx')
# oti = pd.read_excel('/Users/503056565/Data/Customer/source files/OTI.xlsx')
# oti = setup_sii_sc_hierarchy(oti,sii_hierarchy,oti_modality, 'OTI')

# oti.to_csv('/Users/503056565/Data/Customer/scorecard calculations/OTI_Calculations.csv',index=False)
# oti_score_card = setup_scorecard_structure(oti)
# oti_score_card_final = calculate_oti(oti_score_card,oti)
# oti_score_card_final.to_csv('/Users/503056565/Data/Customer/scorecard calculations/OTI_SC.csv',index=False)

# ##CSO (ok. Red open CSO) use Mayo file
# cso_modality = pd.read_excel('/Users/503056565/Data/Customer/configuration/CSO Modalities.xlsx')
# cso = pd.read_excel('/Users/503056565/Data/Customer/source files/CSO.xlsx')
# print(cso.columns)
# cso = setup_sii_sc_hierarchy(cso,sii_hierarchy,cso_modality,'CSO')
# cso.to_csv('/Users/503056565/Data/Customer/scorecard calculations/CSO_Calculations.csv',index=False)
# cso_score_card = setup_scorecard_structure(cso)
# cso_score_card_final = calculate_cso_for_customer(cso_score_card,cso)
# cso_score_card_final.to_csv('/Users/503056565/Data/Customer/scorecard calculations/CSO_SC.csv', index=False)

# ##Complaint (Don't change OSU file). Logic will calculate direct complete rate which is not required for Mayo
# chu_hierarchy =  pd.read_excel('/Users/503056565/Data/Customer/configuration/CHU Hierarchy.xlsx')
# complaint_customer = pd.read_excel('/Users/503056565/Data/Customer/source files/Complaints_OSU_Source_2704.xlsx')
# complaint_customer = setup_chu_hierarchy(complaint_customer, chu_hierarchy)
# complaint_customer.to_csv('/Users/503056565/Data/Customer/scorecard calculations/Complaints_OSU_Calculations.csv',index=False)
# complaint_customer = setup_data_for_cr_calculations(complaint_customer)

# complaint_customer_sc = setup_scorecard_structure(pd.read_csv('/Users/503056565/Data/Customer/scorecard calculations/Complaints_OSU_Calculations.csv'))
# complaint_customer_sc_final = calculate_cpu_rate(complaint_customer_sc,complaint_customer)
# # complaint_customer_sc_final.drop(columns=['Business', 'Modality','Product Segment'], inplace=True)
# complaint_customer_sc_final.to_csv('/Users/503056565/Data/Customer/scorecard calculations/Complaints_OSU_SC.csv', index=False)

# ##IFR (ok)
# ifr_modalities = pd.read_excel('/Users/503056565/Data/Customer/configuration/IFR Modalities.xlsx')
# ifr_customer = pd.read_excel('/Users/503056565/Data/Customer/source files/IFR.xlsx')
# ifr_sc_mapping = setup_ifr_hierarchy(ifr_customer,sii_hierarchy,ifr_modalities)
# ifr_sc_mapping.to_csv('/Users/503056565/Data/Customer/scorecard calculations/IFR_Calculations.csv',index=False)
# ifr_sc = setup_scorecard_structure(ifr_sc_mapping)
# ifr_sc_final = calculate_ifr_rate(ifr_sc,ifr_sc_mapping)
# ifr_sc_final.to_csv('/Users/503056565/Data/Customer/scorecard calculations/IFR_SC.csv',index=False)

# # ##CSO Open Tkts (Don't change OSU file).
# # cso_modality = pd.read_excel('/Users/503056565/Data/Customer/configuration/CSO Modalities.xlsx')
# # cso_o = pd.read_excel('/Users/503056565/Data/Customer/source files/CSO_Open.xlsx')
# # print(cso.columns)
# # cso_o = setup_sii_sc_hierarchy(cso_o,sii_hierarchy,cso_modality,'CSO Open')
# # cso_o.to_csv('/Users/503056565/Data/Customer/scorecard calculations/CSO_Open_Calculations.csv',index=False)
# # cso_score_card = setup_scorecard_structure(cso_o)
# # cso_score_card_final = calculate_cso_open_for_customer(cso_score_card,cso_o)
# # cso_score_card_final.to_csv('/Users/503056565/Data/Customer/scorecard calculations/CSO_Open_SC.csv', index=False)

# #IB (ok)
# ib_data = pd.read_excel('/Users/503056565/Data/Customer/source files/IB.xlsx')
# sii_ib_data = setup_sii_sc_hierarchy_ib(ib_data, sii_hierarchy)
# sii_ib_data.to_csv('/Users/503056565/Data/Customer/scorecard calculations/IB_Calculations.csv',index=False)
# sii_ib_data=pd.read_csv("/Users/503056565/Data/Customer/scorecard calculations/IB_Calculations.csv")
# ib_sc = setup_scorecard_structure(sii_ib_data)
# ib_sc = calculate_ib(ib_sc,sii_ib_data)
# ib_sc.to_csv('/Users/503056565/Data/Customer/scorecard calculations/IB_SC.csv', index=False)

# #Complaints PQM (ok)
# complaint_customer_pqm = pd.read_excel('/Users/503056565/Data/Customer/source files/Complaints_pqm.xlsx')
# # complaint_customer_pqm=complaint_customer_pqm[complaint_customer_pqm["Measure Names"]=="# of Complaints"]
# complaint_customer_pqm = setup_chu_hierarchy(complaint_customer_pqm, chu_hierarchy)
# complaint_customer_pqm.to_csv('/Users/503056565/Data/Customer/scorecard calculations/Complaints_PQM_Calculations.csv',index=False)
# complaint_customer_pqm = setup_data_for_cr_calculations_pqm(complaint_customer_pqm)

# complaint_customer_sc_pqm = setup_scorecard_structure(pd.read_csv('/Users/503056565/Data/Customer/scorecard calculations/Complaints_PQM_Calculations.csv'))
# complaint_customer_sc_final_pqm = calculate_cpu_pqm(complaint_customer_sc_pqm,complaint_customer_pqm)
# #complaint_customer_sc_final.drop(columns=['Business', 'Modality','Product Segment'], inplace=True)
# complaint_customer_sc_final_pqm.to_csv('/Users/503056565/Data/Customer/scorecard calculations/Complaints_PQM_SC.csv', index=False)

# # #OBF (ok)
# # obf = pd.read_excel('/Users/503056565/Data/Customer/source files/OBF.xlsx')
# # obf['Measure Names'] = '# of SRs'
# # print(obf.head())
# # sii_obf = setup_obf_hierarchy(obf, sii_hierarchy, 'OBF')
# # sii_obf.to_csv('/Users/503056565/Data/Customer/scorecard calculations/OBF_Calculations.csv',index=False)
# # sii_obf=pd.read_csv("/Users/503056565/Data/Customer/scorecard calculations/OBF_Calculations.csv")
# # obf_sc = setup_scorecard_structure(sii_obf)
# # obf_sc = calculate_obf(obf_sc,sii_obf)
# # obf_sc.to_csv('/Users/503056565/Data/Customer/scorecard calculations/OBF_SC.csv', index=False)


#ELF (ok)
obf = pd.read_excel('/Users/sowmiyanmorri/Documents/Balance Scorecard/Deepa Scorecard Files/Customer/source files/ELF_Spotfire_data.xlsx')
print(obf.head())
elf_modality = pd.read_excel('/Users/sowmiyanmorri/Documents/Balance Scorecard/Deepa Scorecard Files/Customer/configuration/ELF_adj_modality.xlsx')
sii_obf = setup_sii_sc_hierarchy(obf, sii_hierarchy,elf_modality,'ELF')
sii_obf.to_csv('/Users/sowmiyanmorri/Documents/Balance Scorecard/Deepa Scorecard Files/Customer/scorecard calculations/OBF_Cal.csv',index=False)
sii_obf=pd.read_csv("/Users/sowmiyanmorri/Documents/Balance Scorecard/Deepa Scorecard Files/Customer/scorecard calculations/OBF_Cal.csv")
obf_sc = setup_scorecard_structure(sii_obf)
obf_sc = calculate_elf(obf_sc,sii_obf)
obf_sc.to_csv('/Users/sowmiyanmorri/Documents/Balance Scorecard/Deepa Scorecard Files/Customer/scorecard calculations/OBF_SC.csv', index=False)

done
  A Level Modality Code Corrected Modality for Scorecard  \
0                    AW                               AW   
1                  MICT                               MI   
2                    MR                               MR   
3                    MR                               MR   
4                    MR                               MR   

  Product Identifier for Scorecard    SII Family SII Modality   \
0                           CAWL02            AW            AW   
1                           PHL16F           PET            MI   
2                           MOF013      Value MR            MR   
3                           MSF115  1.5T Segment            MR   
4                           MSF118  1.5T Segment            MR   

  sub family / product group Sub-Modality Time Label  Week Number  \
0                WORKSTATION           AW    2018M10           41   
1                       DSTE          PET    2018M10           41   
2                 Ovation HD  

In [ ]:
workbook = xlsxwriter.Workbook('/Users/503056565/Data/Customer/scorecard calculations/Customer_Scorecard.xlsx')
sc_header_format_details = {
    'bold': 1,
    'border': 1,
    'align': 'center',
    'valign': 'vcenter',
    'fg_color': '#0070C0',
    'font_color': 'white',
    'font': 'GE Inspira',
    'font_size': 12,
    'text_wrap': True}
year_header_format_details = {
    'bold': 1,
    'border': 1,
    'align': 'center',
    'valign': 'vcenter',
    'fg_color': '#0070C0',
    'font_color': 'white',
    'font_size': 12,
    'font': 'GE Inspira',
    'text_wrap': True}
org_format_details = {
    'bold': 1,
    'border': 1,
    'align': 'left',
    'valign': 'vcenter',
    'font_color': '#002060',
    'font_size': 12,
    'font': 'GE Inspira',
    'text_wrap': True}
business_format_details = {
    'bold': 1,
    'border': 1,
    'align': 'left',
    'valign': 'vcenter',
    'fg_color': '#0070C0',
    'font_color': 'white',
    'font': 'GE Inspira Sans',
    'font_size': 12,
    'text_wrap': True}
modality_format_details = {
    'bold': 1,
    'border': 1,
    'align': 'left',
    'valign': 'vcenter',
    'fg_color': '#D9E1F2',
    'font_color': '#002060',
    'font': 'GE Inspira',
    'font_size': 12,
    'text_wrap': True,
    'underline': True}
prod_seg_format_details = {
        'bold': 1,
        'border': 1,
        'align': 'left',
        'valign': 'vcenter',
        'font_color': '#002060',
        'font': 'GE Inspira',
        'text_wrap': True,
        'font_size': 12}
prod_format_details = {
        'bold': 1,
        'border': 1,
        'align': 'right',
        'valign': 'vcenter',
        'font_color': '#002060',
        'font': 'GE Inspira',
        'text_wrap': True,
        'font_size': 10}

scorecard_value_format_details = {
    'border': 1,
    'align': 'center',
    'valign': 'vcenter',
    'font_color': '#002060',
    'font': 'GE Inspira',
    'text_wrap': True,
    'font_size': 11}
business_value_format_details = {
    'border': 1,
    'align': 'center',
    'valign': 'vcenter',
    'fg_color': '#0070C0',
    'font_color': 'white',
    'font': 'GE Inspira Sans',
    'font_size': 11,
    'text_wrap': True}
modality_value_format_details = {
    'border': 1,
    'align': 'center',
    'valign': 'vcenter',
    'fg_color': '#D9E1F2',
    'font_color': '#002060',
    'font': 'GE Inspira',
    'font_size': 11,
    'text_wrap': True}
red_format = workbook.add_format({'bg_color':   '#FFC7CE',
                                  'font_color': '#9C0006'})
# Light yellow fill with dark yellow text.
orange_format = workbook.add_format({'bg_color':   '#FFEB9C',
                                     'font_color': '#9C6500'})
# Green fill with dark green text.
green_format = workbook.add_format({'bg_color':   '#C6EFCE',
                                    'font_color': '#006100'})

worksheet = workbook.add_worksheet('Scorecard')
org_format = workbook.add_format(org_format_details)
business_format = workbook.add_format(business_format_details)
modality_format = workbook.add_format(modality_format_details)
prod_seg_format = workbook.add_format(prod_seg_format_details)
prod_format = workbook.add_format(prod_format_details)
sc_header_format = workbook.add_format(sc_header_format_details)
year_header_format = workbook.add_format(year_header_format_details)
scorecard_value_format = workbook.add_format(scorecard_value_format_details)
business_value_format = workbook.add_format(business_value_format_details)
modality_value_format = workbook.add_format(modality_value_format_details)
worksheet.set_column('A:A', 56)
worksheet.set_column('B:AD', 12)

worksheet.set_row(0, 35)
worksheet.merge_range('A1:A2', 'Product Quality Scorecard', sc_header_format)

# # Complaints
# worksheet.merge_range('E1:K1', 'Complaints', sc_header_format)
# worksheet.write('E2', '2018' + '\n' + 'Total*', year_header_format)
# worksheet.write('F2', '2018' + '\n' + 'YTD', year_header_format)
# worksheet.write('G2', '2018' + '\n' + 'YTD IB Installs', year_header_format)
# worksheet.write('H2', '2019' + '\n' + 'YTD', year_header_format)
# worksheet.write('I2', '2019' + '\n' + 'YTD IB Installs', year_header_format)
# worksheet.write('J2', '2018' + '\n' + 'YTD IB Complaints', year_header_format)
# worksheet.write('K2', '2019' + '\n' + 'YTD IB Complaints', year_header_format)
# worksheet.conditional_format('H3:H2250', {'type':'formula',
#                                        'criteria':'=IF(H3="n/a",0,H3)<IF(F3="n/a",0,F3)',
#                                        'format':green_format})
# # worksheet.conditional_format('H3:H2250',{'type': 'icon_set',
# #                                          'icon_style': '3_arrows','icons': [{'criteria': '<', 'type': 'formula','IF(F3="n/a",0,F3)'},
# #                                                    {'criteria': '=',  'type': 'formula', 'IF(F3="n/a",0,F3)'},
# #                                                    {'criteria': '>', 'type': 'formula',    'IF(F3="n/a",0,F3)'}]})
# worksheet.conditional_format('H3:H2250', {'type':'formula',
#                                        'criteria':'=IF(H3="n/a",0,H3)=IF(F3="n/a",0,F3)',
#                                        'format':orange_format})
# worksheet.conditional_format('H3:H2250', {'type':'formula',
#                                        'criteria':'=IF(H3="n/a",0,H3)>IF(F3="n/a",0,F3)',
#                                        'format':red_format})
# On Time delivery
worksheet.merge_range('E1:I1', 'On Time Delivery', sc_header_format)
worksheet.write('E2', '2018' + '\n' + 'Total*', year_header_format)
worksheet.write('F2', '2018' + '\n' + 'YTD', year_header_format)
worksheet.write('G2', '2018' + '\n' + 'YTD Installs', year_header_format)
worksheet.write('H2', '2019' + '\n' + 'YTD', year_header_format)
worksheet.write('I2', '2019' + '\n' + 'YTD Installs', year_header_format)


worksheet.conditional_format('I3:I2250', {'type':'formula',
                                       'criteria':'=IF(I3="n/a",0,I3)<IF(F3="n/a",0,F3)',
                                       'format':red_format})
worksheet.conditional_format('I3:I2250', {'type':'formula',
                                       'criteria':'=IF(I3="n/a",0,I3)=IF(F3="n/a",0,F3)',
                                       'format':orange_format})
worksheet.conditional_format('I3:I2250', {'type':'formula',
                                       'criteria':'=IF(I3="n/a",0,I3)>IF(F3="n/a",0,F3)',
                                       'format':green_format})



# On Time Install
worksheet.merge_range('J1:N1', 'On Time Install', sc_header_format)
worksheet.write('J2', '2018' + '\n' + 'Total*', year_header_format)
worksheet.write('K2', '2018' + '\n' + 'YTD', year_header_format)
worksheet.write('L2', '2018' + '\n' + 'YTD Installs', year_header_format)
worksheet.write('M2', '2019' + '\n' + 'YTD', year_header_format)
worksheet.write('N2', '2019' + '\n' + 'YTD Installs', year_header_format)


worksheet.conditional_format('M3:M2250', {'type':'formula',
                                       'criteria':'=IF(M3="n/a",0,M3)<IF(K3="n/a",0,K3)',
                                       'format':red_format})
worksheet.conditional_format('M3:M2250', {'type':'formula',
                                       'criteria':'=IF(M3="n/a",0,M3)=IF(K3="n/a",0,K3)',
                                       'format':orange_format})
worksheet.conditional_format('M3:M2250', {'type':'formula',
                                       'criteria':'=IF(M3="n/a",0,M3)>IF(K3="n/a",0,K3)',
                                       'format':green_format})


# IFR90
worksheet.merge_range('O1:S1', 'IFR90', sc_header_format)
worksheet.write('O2', '2018' + '\n' + 'Total*', year_header_format)
worksheet.write('P2', '2018' + '\n' + 'YTD', year_header_format)
worksheet.write('Q2', '2018' + '\n' + 'YTD Installs', year_header_format)
worksheet.write('R2', '2019' + '\n' + 'YTD', year_header_format)
worksheet.write('S2', '2019' + '\n' + 'YTD Installs', year_header_format)


worksheet.conditional_format('R3:R2250', {'type':'formula',
                                       'criteria':'=IF(R3="n/a",0,R3)<IF(P3="n/a",0,P3)',
                                       'format':green_format})
worksheet.conditional_format('R3:R2250', {'type':'formula',
                                       'criteria':'=IF(R3="n/a",0,R3)=IF(P3="n/a",0,P3)',
                                       'format':orange_format})
worksheet.conditional_format('R3:R2250', {'type':'formula',
                                       'criteria':'=IF(R3="n/a",0,R3)>IF(P3="n/a",0,P3)',
                                       'format':red_format})

# CSO
worksheet.merge_range('T1:U1', 'CSOs (# Open Red > 90 days)', sc_header_format)
worksheet.write('T2', '2018' + '\n' + 'Total*', year_header_format)
worksheet.write('U2', '2019' + '\n' + 'YTD', year_header_format)

worksheet.conditional_format('U3:U2250', {'type':'formula',
                                       'criteria':'=IF(U3="n/a",0,U3)<IF(T3="n/a",0,T3)',
                                       'format':green_format})
worksheet.conditional_format('U3:U2250', {'type':'formula',
                                       'criteria':'=IF(U3="n/a",0,U3)=IF(T3="n/a",0,T3)',
                                       'format':orange_format})
worksheet.conditional_format('U3:U2250', {'type':'formula',
                                       'criteria':'=IF(U3="n/a",0,U3)>IF(T3="n/a",0,T3)',
                                      'format':red_format})

# # CSO Open
# worksheet.merge_range('AC1:AE1', 'CSO', sc_header_format)
# worksheet.write('AC2', '2018' + '\n' + 'Total', year_header_format)
# worksheet.write('AD2', '2018' + '\n' + 'YTD', year_header_format)
# worksheet.write('AE2', '2019' + '\n' + 'YTD', year_header_format)

# IB
worksheet.merge_range('V1:X1', 'IB Details', sc_header_format)
worksheet.write('V2', '2018' + '\n' + 'IB Total', year_header_format)
worksheet.write('W2', '2018' + '\n' + 'IB YTD', year_header_format)
worksheet.write('X2', '2019' + '\n' + 'IB YTD', year_header_format)

# Comp_pqm
worksheet.merge_range('Y1:AA1', 'Complaints Details', sc_header_format)
worksheet.write('Y2', '2018' + '\n' + 'Complaint Total', year_header_format)
worksheet.write('Z2', '2018' + '\n' + 'Complaint YTD', year_header_format)
worksheet.write('AA2', '2019' + '\n' + 'Complaint YTD', year_header_format)

worksheet.conditional_format('AA3:AA2250', {'type':'formula',
                                       'criteria':'=IF(AA3="n/a",0,AA3)<IF(Z3="n/a",0,Z3)',
                                       'format':green_format})
worksheet.conditional_format('AA3:AA2250', {'type':'formula',
                                       'criteria':'=IF(AA3="n/a",0,AA3)=IF(Z3="n/a",0,Z3)',
                                       'format':orange_format})
worksheet.conditional_format('AA3:AA2250', {'type':'formula',
                                       'criteria':'=IF(AA3="n/a",0,AA3)>IF(Z3="n/a",0,Z3)',
                                      'format':red_format})

# OBF
worksheet.merge_range('AB1:AD1', 'OBF', sc_header_format)
worksheet.write('AB2', '2018' + '\n' + 'OBF Total', year_header_format)
worksheet.write('AC2', '2018' + '\n' + 'OBF YTD', year_header_format)
worksheet.write('AD2', '2019' + '\n' + 'OBF YTD', year_header_format)

worksheet.conditional_format('AD3:AD2250', {'type':'formula',
                                       'criteria':'=IF(AD3="n/a",0,AD3)<IF(AC3="n/a",0,AC3)',
                                       'format':green_format})
worksheet.conditional_format('AD3:AD2250', {'type':'formula',
                                       'criteria':'=IF(AD3="n/a",0,AD3)=IF(AC3="n/a",0,AC3)',
                                       'format':orange_format})
worksheet.conditional_format('AD3:AD2250', {'type':'formula',
                                       'criteria':'=IF(AD3="n/a",0,AD3)>IF(AC3="n/a",0,AC3)',
                                      'format':red_format})

# OTI total install
# worksheet.merge_range('S1:U1', 'Total OTI Install', sc_header_format)
# worksheet.write('S2', '2018' + '\n' + 'Total*', year_header_format)
# worksheet.write('T2', '2018' + '\n' + 'YTD', year_header_format)
# worksheet.write('U2', '2019' + '\n' + 'YTD', year_header_format)



In [ ]:
complaints = pd.read_csv('/Users/503056565/Data/Customer/scorecard calculations/Complaints_SC.csv')
cso = pd.read_csv('/Users/503056565/Data/Customer/scorecard calculations/CSO_SC.csv')
# cso_o = pd.read_csv('/Users/503056565/Data/Customer/scorecard calculations/CSO_Open_SC.csv')
otd = pd.read_csv('/Users/503056565/Data/Customer/scorecard calculations/OTD_SC.csv')
oti = pd.read_csv('/Users/503056565/Data/Customer/scorecard calculations/OTI_SC.csv')
ifr = pd.read_csv('/Users/503056565/Data/Customer/scorecard calculations/IFR_SC.csv')
ib = pd.read_csv('/Users/503056565/Data/Customer/scorecard calculations/IB_SC.csv')
comp = pd.read_csv('/Users/503056565/Data/Customer/scorecard calculations/Complaints_PQM_SC.csv')
obf = pd.read_csv('/Users/503056565/Data/Customer/scorecard calculations/OBF_SC.csv')

In [ ]:
complaints_sc = complaints[['Product Quality Scorecard','Business','Modality','Product Segment']]
cso_sc = cso[['Product Quality Scorecard','Business','Modality','Product Segment']]
# cso_open_sc = cso_o[['Product Quality Scorecard','Business','Modality','Product Segment']]
otd_sc = otd[['Product Quality Scorecard','Business','Modality','Product Segment']]
oti_sc = oti[['Product Quality Scorecard','Business','Modality','Product Segment']]
ifr_sc = ifr[['Product Quality Scorecard','Business','Modality','Product Segment']]
ib_sc = ib[['Product Quality Scorecard','Business','Modality','Product Segment']]
comp_sc = comp[['Product Quality Scorecard','Business','Modality','Product Segment']]
obf_sc = obf[['Product Quality Scorecard','Business','Modality','Product Segment']]

#combined_sc = complaints_sc.append(cso_sc).append(otd_sc).append(oti_sc).append(ifr_sc).append(ib_sc)
combined_sc = cso_sc.append(otd_sc).append(oti_sc).append(ifr_sc).append(ib_sc).append(comp_sc).append(obf_sc)
combined_sc.drop_duplicates(inplace=True)
combined_sc.reset_index(inplace=True)
combined_sc.to_csv('/Users/503056565/Data/Customer/scorecard calculations/Combined_SC.csv')


In [ ]:
sc_hierarchy = combined_sc['Business'].unique()
worksheet.write('A3', 'GEHC', org_format)
worksheet.write('B3' + 'All', 'All', prod_format)
worksheet.write('C3' + 'All', 'All', prod_format)
worksheet.write('D3' + 'All', 'All', prod_format)
index=4
for row in sc_hierarchy:
    if row !='All':
        worksheet.write('A' + str(index), row, business_format)
        worksheet.write('B' + str(index), 'All', prod_format)
        worksheet.write('C' + str(index), 'All', prod_format)
        worksheet.write('D' + str(index), 'All', prod_format)
        index = index + 1
        modalities = combined_sc.loc[combined_sc['Business'] == row]['Modality'].unique()
        for modality in modalities:
            if modality != 'All':
                worksheet.write('A' + str(index), modality, modality_format)
                worksheet.write('B' + str(index), row, prod_format)
                worksheet.write('C' + str(index), 'All', prod_format)
                worksheet.write('D' + str(index), 'All', prod_format)
                index = index + 1
                prod_segments = combined_sc.loc[(combined_sc['Business'] == row) & (combined_sc['Modality'] == modality)]['Product Segment'].unique()
                for prod_segment in prod_segments:
                    if prod_segment != 'All':
                        worksheet.write('A' + str(index), prod_segment, prod_seg_format)
                        worksheet.write('B' + str(index), row, prod_format)
                        worksheet.write('C' + str(index), modality, prod_format)
                        worksheet.write('D' + str(index), 'All', prod_format)
                        index = index + 1
#                         products = combined_sc.loc[(combined_sc['Business'] == row) & (combined_sc['Modality'] == modality) & \
#                                                   (combined_sc['Product Segment'] == prod_segment)]['Product Quality Scorecard']\
#                                                         .unique()

#                         for product in products:
#                             worksheet.write('A' + str(index), product, prod_format)
#                             worksheet.write('B' + str(index), row, prod_format)
#                             worksheet.write('C' + str(index), modality, prod_format)
#                             worksheet.write('D' + str(index), prod_segment, prod_format)
#                             index = index + 1

#complaints.fillna('n/a',inplace=True)
#ib.fillna('n/a',inplace=True)



# fill_in_sc_data(combined_sc, scorecard_value_format, business_value_format, modality_value_format,
#                                       complaints, otd, oti, cso,cso_o, worksheet,ifr)


In [ ]:
fill_in_sc_data(combined_sc, scorecard_value_format, business_value_format, modality_value_format,
                                      otd, oti, cso, worksheet, ifr, ib, comp, obf)

In [ ]:
fill_in_sc_data(combined_sc, scorecard_value_format, business_value_format, modality_value_format,
                                      otd, oti, cso, worksheet, ifr, ib, comp, obf)